In [ ]:
# | default_exp mkdocs

In [ ]:
# | export

import collections
import datetime
import importlib
import multiprocessing
import os
import pkgutil
import re
import shlex
import shutil
import subprocess  # nosec: B404
import sys
import textwrap
import types
from configparser import ConfigParser
from pathlib import Path
from typing import *

import nbdev
import typer
import yaml
from configupdater import ConfigUpdater, Section
from configupdater.option import Option
from fastcore.shutil import move
from nbdev.doclinks import nbdev_export
from nbdev.frontmatter import FrontmatterProc, _fm2dict
from nbdev.process import NBProcessor
from nbdev.quarto import nbdev_readme
from nbdev.quarto import prepare as nbdev_prepare
from nbdev.quarto import refresh_quarto_yml
from nbdev.serve import proc_nbs

from nbdev_mkdocs._helpers.cli_doc import generate_cli_doc
from nbdev_mkdocs._helpers.utils import get_value_from_config, set_cwd
from nbdev_mkdocs._package_data import get_root_data_path
from nbdev_mkdocs.social_image_generator import _update_social_image_in_mkdocs_yml

In [ ]:
import json
import unittest.mock
from contextlib import contextmanager
from tempfile import TemporaryDirectory

import numpy as np
import pytest
from fastcore.imports import IN_NOTEBOOK
from nbdev.config import nbdev_create_config
from ruamel.yaml import YAML

## Create new

### Create mkdocs dir

In [ ]:
# | export


def _create_mkdocs_dir(root_path: str) -> None:
    """Create a mkdocs directory in the root path.

    Args:
        root_path: The root path.

    Returns:
        None

    Raises:
        typer.Exit: If the mkdocs_template path does not exist.

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
    """
    mkdocs_template_path = get_root_data_path() / "mkdocs_template"
    if not mkdocs_template_path.exists():
        typer.secho(
            f"Unexpected error: path {mkdocs_template_path.resolve()} does not exists!",
            err=True,
            fg=typer.colors.RED,
        )
        raise typer.Exit(code=4)
    dst_path = Path(root_path) / "mkdocs"
    if dst_path.exists():
        typer.secho(
            f"Directory {dst_path.resolve()} already exist, skipping its creation.",
        )
    else:
        shutil.copytree(mkdocs_template_path, dst_path)
        #         shutil.move(dst_path.parent / "mkdocs_template", dst_path)
        typer.secho(
            f"Directory {dst_path.resolve()} created.",
        )

In [ ]:
with TemporaryDirectory() as d:
    settings_path = Path(d) / "settings.ini"
    shutil.copyfile(Path("..") / "settings.ini", settings_path)

    _create_mkdocs_dir(d)

    print("\n".join([str(p) for p in (Path(d) / "mkdocs").glob("**/*")]))

Directory /private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpbzx65z6g/mkdocs created.
/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpbzx65z6g/mkdocs/site_overrides
/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpbzx65z6g/mkdocs/.ipynb_checkpoints
/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpbzx65z6g/mkdocs/docs_overrides
/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpbzx65z6g/mkdocs/site_overrides/main.html
/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpbzx65z6g/mkdocs/site_overrides/partials
/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpbzx65z6g/mkdocs/site_overrides/partials/copyright.html
/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpbzx65z6g/mkdocs/docs_overrides/css
/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpbzx65z6g/mkdocs/docs_overrides/images
/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpbzx65z6g/mkdocs/docs_overrides/js
/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpbzx65z6g/mkdocs/docs_overrides/css/extra.css


### Create Mkdocs.yml

In [ ]:
# | export

_mkdocs_template_path = get_root_data_path() / "mkdocs_template.yml"

In [ ]:
assert _mkdocs_template_path.exists()

In [ ]:
# | export

with open(_mkdocs_template_path, "r") as f:
    _mkdocs_template = f.read()

In [ ]:
print(_mkdocs_template)

# Site
site_name: {title}
site_url: {doc_host}{doc_baseurl}
site_author: {author}
site_description: {description}
  
# Repository
repo_name: {repo}
repo_url: {git_url}
edit_uri: ""

copyright: {copyright}

docs_dir: docs
site_dir: site

plugins:
- literate-nav:
    nav_file: SUMMARY.md
- search
- mkdocstrings:
    handlers:
      python:
        import:
            - https://docs.python.org/3/objects.inv
        options:
            heading_level: 2
            show_category_heading: true
            show_root_heading: true
            show_root_toc_entry: true
            show_signature_annotations: true
            show_if_no_docstring: true
            
markdown_extensions:
    - md_in_html
    - pymdownx.arithmatex:
        generic: true
    - pymdownx.inlinehilite
    - pymdownx.details
    - pymdownx.emoji
    - pymdownx.magiclink
    - pymdownx.superfences:
        custom_fences:
          - name: mermaid
            class: mermaid
            format: !!python/name:pymdownx.supe

In [ ]:
# | export
def _get_kwargs_from_settings(
    settings_path: Path, mkdocs_template: Optional[str] = None
) -> Dict[str, str]:
    """Get the values from the settings file

    Args:
        settings_path: The path to the settings file
        mkdocs_template: The mkdocs template to use

    Returns:
        A dictionary of the kwargs

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
    """
    config = ConfigParser()
    config.read(settings_path)
    if not mkdocs_template:
        mkdocs_template = _mkdocs_template
    keys = [s[1:-1] for s in re.findall("\{.*?\}", _mkdocs_template)]
    kwargs = {k: config["DEFAULT"][k] for k in keys}
    return kwargs

In [ ]:
with TemporaryDirectory() as d:
    settings_path = Path(d) / "settings.ini"
    shutil.copyfile(Path("..") / "settings.ini", settings_path)

    kwargs = _get_kwargs_from_settings(settings_path)

    actual = _mkdocs_template.format(**kwargs)

kwargs

{'title': 'nbdev-mkdocs',
 'doc_host': 'https://airtai.github.io',
 'doc_baseurl': '/nbdev-mkdocs',
 'author': 'airt',
 'description': 'Extension of nbdev for generating documentation using Material for Mkdocs instead of Quarto',
 'repo': 'nbdev-mkdocs',
 'git_url': 'https://github.com/airtai/nbdev-mkdocs',
 'copyright': '2022 onwards, airt'}

In [ ]:
# | export


def _create_mkdocs_yaml(root_path: str) -> None:
    """Create mkdocs.yml file

    Args:
        root_path: The root path of the project

    Raises:
        ValueError: If root_path is invalid or does not exists

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
    """
    try:
        # create mkdocs folder if necessary
        mkdocs_path = Path(root_path) / "mkdocs" / "mkdocs.yml"
        mkdocs_path.parent.mkdir(exist_ok=True)
        # mkdocs.yml already exists, just return
        if mkdocs_path.exists():
            typer.secho(
                f"Path '{mkdocs_path.resolve()}' exists, skipping generation of it."
            )
            return

        # get default values from settings.ini
        settings_path = Path(root_path) / "settings.ini"
        kwargs = _get_kwargs_from_settings(settings_path)
        mkdocs_yaml_str = _mkdocs_template.format(**kwargs)
        with open(mkdocs_path, "w") as f:
            f.write(mkdocs_yaml_str)
            typer.secho(f"File '{mkdocs_path.resolve()}' generated.")
            return
    except Exception as e:
        typer.secho(
            f"Unexpected Error while creating '{mkdocs_path.resolve()}': {e}",
            err=True,
            fg=typer.colors.RED,
        )
        raise typer.Exit(code=3)

In [ ]:
with TemporaryDirectory() as d:
    settings_path = Path(d) / "settings.ini"
    shutil.copyfile(Path("..") / "settings.ini", settings_path)

    _create_mkdocs_yaml(d)

#     !cat {d}/mkdocs/mkdocs.yml

File '/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpvrvf5zbu/mkdocs/mkdocs.yml' generated.


### Create summary_template.txt

In [ ]:
# | export

_summary_template = """{sidebar}
- API
{api}
- CLI
{cli}
- [Releases]{changelog}
"""


def _create_summary_template(root_path: str) -> None:
    """Create a summary template file for mkdocs.

    Args:
        root_path: The root path of the project.

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
    """
    try:
        # create mkdocs folder if necessary
        summary_template_path = Path(root_path) / "mkdocs" / "summary_template.txt"
        summary_template_path.parent.mkdir(exist_ok=True)
        # summary_template_path.yml already exists, just return
        if summary_template_path.exists():
            typer.secho(
                f"Path '{summary_template_path.resolve()}' exists, skipping generation of it."
            )
            return

        # generated a new summary_template_path.yml file
        with open(summary_template_path, "w") as f:
            f.write(_summary_template)
            typer.secho(f"File '{summary_template_path.resolve()}' generated.")
            return
    except Exception as e:
        typer.secho(
            f"Unexpected Error while creating '{summary_template_path.resolve()}': {e}",
            err=True,
            fg=typer.colors.RED,
        )
        raise typer.Exit(code=3)

In [ ]:
with TemporaryDirectory() as d:
    settings_path = Path(d) / "settings.ini"
    shutil.copyfile(Path("..") / "settings.ini", settings_path)

    _create_mkdocs_yaml(d)
    _create_summary_template(d)

    sidebar = """- [Home](index.md)
- Guides
    - [Guide one](docs/guide_1.md)
- Tutorial
    - [Tutorial one](docs/tutorial_1.md)"""

    api = """    - [numpy.array](api/numpy/array.md)"""

    cli = """    - [my-cli](cli/my_cli.md)"""

    changelog = "(CHANGELOG.md)"

    with open(Path(d) / "mkdocs/summary_template.txt") as f:
        summary_template = f.read()
        actual = summary_template.format(
            sidebar=sidebar, api=api, cli=cli, changelog=changelog
        )

expected = """- [Home](index.md)
- Guides
    - [Guide one](docs/guide_1.md)
- Tutorial
    - [Tutorial one](docs/tutorial_1.md)
- API
    - [numpy.array](api/numpy/array.md)
- CLI
    - [my-cli](cli/my_cli.md)
- [Releases](CHANGELOG.md)
"""
print(actual)

assert actual == expected, actual

File '/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpap30uedj/mkdocs/mkdocs.yml' generated.
File '/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpap30uedj/mkdocs/summary_template.txt' generated.
- [Home](index.md)
- Guides
    - [Guide one](docs/guide_1.md)
- Tutorial
    - [Tutorial one](docs/tutorial_1.md)
- API
    - [numpy.array](api/numpy/array.md)
- CLI
    - [my-cli](cli/my_cli.md)
- [Releases](CHANGELOG.md)



In [ ]:
# | export


def _replace_ghp_deploy_action(root_path: str) -> None:
    """Replace the default deploy action file in the .github/workflows directory with a custom one.

    Args:
        root_path: The root path of the project.

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
    """
    src_path = get_root_data_path() / "ghp_deploy_action_template.yml"
    if not src_path.exists():
        typer.secho(
            f"Unexpected error: path {src_path.resolve()} does not exists!",
            err=True,
            fg=typer.colors.RED,
        )
        raise typer.Exit(code=4)

    workflows_path = Path(root_path) / ".github" / "workflows"
    workflows_path.mkdir(exist_ok=True, parents=True)

    dst_path = Path(workflows_path) / "deploy.yaml"
    shutil.copyfile(src_path, dst_path)

In [ ]:
with TemporaryDirectory() as d:
    assert not (Path(d) / ".github" / "workflows" / "deploy.yaml").exists()
    _replace_ghp_deploy_action(d)
    assert (Path(d) / ".github" / "workflows" / "deploy.yaml").exists()

In [ ]:
@contextmanager
def unset_env_var(name: str):
    """Unset an environment variable.

    Args:
        name: The name of the environment variable to unset.

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
    """
    in_test_defined = name in os.environ
    if in_test_defined:
        original_value = os.environ.get(name)
        del os.environ[name]
    try:
        yield
    finally:
        if in_test_defined:
            os.environ[name] = original_value


def run_nbdev_new(d):
    """Run nbdev_new

    Args:
        d: the directory to run nbdev_new in

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
    """
    path = Path(".") if Path("settings.ini").exists() else Path("..")
    Path(Path(d) / "nbs").mkdir(exist_ok=True)

    # Create a sample .gitignore file
    gitignore_path = Path(d) / ".gitignore"
    sample_git_ignore = (
        "_docs/"
        "_proc/\n"
        "*.bak\n"
        "# Byte-compiled / optimized / DLL files\n"
        "__pycache__/"
    )

    with gitignore_path.open("w", encoding="utf-8") as f:
        f.write(sample_git_ignore)

    assert gitignore_path.exists()

    # copy nbdev-mkdocs/nbs/index.ipynb folder to {d}/nbs/index.ipynb
    for fname in ["index.ipynb"]:
        shutil.copyfile(path / "nbs" / fname, Path(d) / "nbs" / fname)

    # copy config files from nbdev-mkdocs/ to {d}/
    for fname in ["setup.py"]:
        shutil.copyfile(path / fname, Path(d) / fname)

    # Generate settings.ini using nbdev in {d}
    with set_cwd(d):
        nbdev_create_config(
            repo="repo",
            branch="branch",
            user="user",
            author="author",
            author_email="author@mail.com",
            description="description",
        )
        refresh_quarto_yml()

        with unset_env_var("IN_TEST"):
            nbdev_export.__wrapped__()

        nbdev_readme.__wrapped__(chk_time=True)

In [ ]:
# | export


def _update_gitignore_file(root_path: str) -> None:
    """Add the autogenerated mkdocs directories to the .gitignore file.

    Args:
        root_path: The root path of the project

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
    """
    _mkdocs_gitignore_path = get_root_data_path() / "gitignore.txt"
    with open(_mkdocs_gitignore_path, "r") as f:
        _new_paths_to_ignore = f.read()
        _new_paths_to_ignore = "\n\n" + _new_paths_to_ignore

    gitignore_path = Path(root_path) / ".gitignore"
    if not gitignore_path.exists():
        typer.secho(
            f"Unexpected error: path {gitignore_path.resolve()} does not exists!",
            err=True,
            fg=typer.colors.RED,
        )
        raise typer.Exit(code=1)

    with open(gitignore_path, "a") as f:
        f.write(_new_paths_to_ignore)

In [ ]:
with TemporaryDirectory() as d:
    run_nbdev_new(d)
    assert (Path(d) / "repo").exists()

    gitignore_path = Path(d) / ".gitignore"

    with open(gitignore_path) as f:
        contents = f.read()
        assert "mkdocs/docs/" not in contents
        assert "mkdocs/site/" not in contents

    _update_gitignore_file(d)

    with open(gitignore_path) as f:
        contents = f.read()
        print(contents)
        assert "mkdocs/docs/" in contents
        assert "mkdocs/site/" in contents

settings.ini created.


pandoc -o README.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  output-file: index.html
  standalone: true
  default-image-extension: png
  
metadata
  title: Material for nbdev
  


_docs/_proc/
*.bak
# Byte-compiled / optimized / DLL files
__pycache__/

# nbdev_mkdocs
mkdocs/docs/
mkdocs/site/


Output created: _docs/README.md



In [ ]:
# | export


def _generate_default_social_image_link(root_path: str) -> None:
    """Add default social sharing image link to the mkdocs yaml file

    Args:
        root_path: The root path of the project.

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
    """
    with set_cwd(root_path):
        repo = get_value_from_config(root_path, "repo")
        user = get_value_from_config(root_path, "user")

        timestamp = datetime.datetime.now().timestamp()
        img_url = f"https://opengraph.githubassets.com/{timestamp}/{user}/{repo}"

        _update_social_image_in_mkdocs_yml(root_path, img_url)

In [ ]:
with TemporaryDirectory() as d:

    run_nbdev_new(d)

    _create_mkdocs_dir(d)
    _create_mkdocs_yaml(d)
    _create_summary_template(d)
    _replace_ghp_deploy_action(d)
    _update_gitignore_file(d)

    _generate_default_social_image_link(d)

    _yaml = YAML()
    y = _yaml.load(Path(d) / "mkdocs/mkdocs.yml")
    print(y)
    assert y["extra"]["social_image"] != ""

settings.ini created.


pandoc -o README.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  output-file: index.html
  standalone: true
  default-image-extension: png
  
metadata
  title: Material for nbdev
  


Directory /private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpo7zfiq32/mkdocs created.
File '/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpo7zfiq32/mkdocs/mkdocs.yml' generated.
File '/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpo7zfiq32/mkdocs/summary_template.txt' generated.
ordereddict([('site_name', 'repo'), ('site_url', 'https://user.github.io/repo'), ('site_author', 'author'), ('site_description', 'description'), ('repo_name', 'repo'), ('repo_url', 'https://github.com/user/repo'), ('edit_uri', ''), ('copyright', '2023 onwards, author'), ('docs_dir', 'docs'), ('site_dir', 'site'), ('plugins', [ordereddict([('literate-nav', ordereddict([('nav_file', 'SUMMARY.md')]))]), 'search', ordereddict([('mkdocstrings', ordereddict([('handlers', ordereddict([('python', ordereddict([('import', ['https://docs.python.org/3/objects.inv']), ('options', ordereddict([('heading_level', 2), ('show_category_heading', True), ('show_root_heading', True), ('show_root_to

Output created: _docs/README.md



### Bringing it all together

In [ ]:
# | export


def new(root_path: str) -> None:
    """Initialize mkdocs project files

    Creates **mkdocs** directory in the **root_path** directory and populates
    it with initial values. You should edit mkdocs.yml file to customize it if
    needed.

    Args:
        root_path: The path to the root of the project

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
    """
    _create_mkdocs_dir(root_path)
    _create_mkdocs_yaml(root_path)
    _create_summary_template(root_path)
    _replace_ghp_deploy_action(root_path)
    _update_gitignore_file(root_path)
    _generate_default_social_image_link(root_path)


In [ ]:
with TemporaryDirectory() as d:
    run_nbdev_new(d)

    settings_path = Path(d) / "settings.ini"
    assert settings_path.exists()

    new(d)

    mkdocs_path = Path(d) / "mkdocs"

    assert mkdocs_path.exists()
    assert (mkdocs_path / "mkdocs.yml").exists()
    assert (mkdocs_path / "site_overrides" / "main.html").exists()
    assert (mkdocs_path / "site_overrides" / "partials" / "copyright.html").exists()
    assert (mkdocs_path / "summary_template.txt").exists()

    gitignore_path = Path(d) / ".gitignore"
    with open(gitignore_path) as f:
        contents = f.read()
        print(contents)
        assert "mkdocs/docs/" in contents
        assert "mkdocs/site/" in contents

    _yaml = YAML()
    y = _yaml.load(Path(d) / "mkdocs/mkdocs.yml")
    print(y)
    assert y["extra"]["social_image"] != ""

settings.ini created.


pandoc -o README.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  output-file: index.html
  standalone: true
  default-image-extension: png
  
metadata
  title: Material for nbdev
  


Directory /private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp4hjguehz/mkdocs created.
File '/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp4hjguehz/mkdocs/mkdocs.yml' generated.
File '/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp4hjguehz/mkdocs/summary_template.txt' generated.
_docs/_proc/
*.bak
# Byte-compiled / optimized / DLL files
__pycache__/

# nbdev_mkdocs
mkdocs/docs/
mkdocs/site/
ordereddict([('site_name', 'repo'), ('site_url', 'https://user.github.io/repo'), ('site_author', 'author'), ('site_description', 'description'), ('repo_name', 'repo'), ('repo_url', 'https://github.com/user/repo'), ('edit_uri', ''), ('copyright', '2023 onwards, author'), ('docs_dir', 'docs'), ('site_dir', 'site'), ('plugins', [ordereddict([('literate-nav', ordereddict([('nav_file', 'SUMMARY.md')]))]), 'search', ordereddict([('mkdocstrings', ordereddict([('handlers', ordereddict([('python', ordereddict([('import', ['https://docs.python.org/3/objects.inv']), ('options

Output created: _docs/README.md



## Build

### Build markdown files

In [ ]:
# | export


def _get_files_to_convert_to_markdown(cache: Path) -> List[Path]:
    """Get a list of notebooks and qmd files that require conversion to markdown format.

    Args:
        cache: The cache directory path

    Returns:
        A list of files to convert to markdown

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
    """
    exts = [".ipynb", ".qmd"]
    files = [
        f
        for f in cache.rglob("*")
        if f.suffix in exts and not any(p.startswith(".") for p in f.parts)
    ]

    return files

In [ ]:
def create_sample_qmd_file(d):
    """Create a sample qmd file

    Args:
        d: Path to the directory where the file should be created

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
    """
    qmd_str = """---
title: "Sample"
author: Sample
date: last-modified
---

## Introduction

"""
    qmd_index_path = Path(d) / "nbs" / "sample.qmd"
    with open(qmd_index_path, "w") as f:
        f.write(qmd_str)


with TemporaryDirectory() as d:

    run_nbdev_new(d)

    (Path(d) / "_proc" / "guides").mkdir(exist_ok=True)
    (Path(d) / "_proc" / "blogs").mkdir(exist_ok=True)

    _nbs_path = (
        Path(".") / "nbs" / "Mkdocs.ipynb"
        if Path("settings.ini").exists()
        else Path("..") / "nbs" / "Mkdocs.ipynb"
    )
    shutil.copyfile(_nbs_path, Path(d) / "_proc" / "Mkdocs.ipynb")

    create_sample_qmd_file(d)

    for i in ["guides", "blogs"]:
        (Path(d) / "_proc" / f"{i}" / ".ipynb_checkpoints").mkdir(exist_ok=True)
        shutil.copyfile(
            Path(d) / "nbs" / "index.ipynb",
            Path(d) / "_proc" / f"{i}" / f"{i}_index.ipynb",
        )
        shutil.copyfile(
            Path(d) / "nbs" / "sample.qmd",
            Path(d) / "_proc" / f"{i}" / f"qmd_{i}.qmd",
        )
        shutil.copyfile(
            Path(d) / "nbs" / "index.ipynb",
            Path(d) / "_proc" / f"{i}" / ".ipynb_checkpoints" / f"{i}_index.ipynb",
        )
        shutil.copyfile(
            Path(d) / "nbs" / "index.ipynb",
            Path(d) / "_proc" / f"{i}" / f"_{i}_index.ipynb",
        )

    new(d)

    with set_cwd(d):
        nbs = _get_files_to_convert_to_markdown(Path(d))

    nbs = [str(nb) for nb in nbs]
    print(nbs)

    assert f"{d}/_proc/index.ipynb" in nbs
    assert f"{d}/_proc/guides/guides_index.ipynb" in nbs
    assert f"{d}/_proc/blogs/qmd_blogs.qmd" in nbs
    assert f"{d}/_proc/Mkdocs.ipynb" in nbs

    assert f"{d}/_proc/_quarto.yml" not in nbs
    assert f"{d}/_proc/guides/.ipynb_checkpoints/guides_index.ipynb" not in nbs
    assert f"{d}/_proc/blogs/.ipynb_checkpoints/blogs_index.ipynb" not in nbs

settings.ini created.


pandoc -o README.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  output-file: index.html
  standalone: true
  default-image-extension: png
  
metadata
  title: Material for nbdev
  


Directory /private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpyfpwne2c/mkdocs created.
File '/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpyfpwne2c/mkdocs/mkdocs.yml' generated.
File '/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpyfpwne2c/mkdocs/summary_template.txt' generated.
['/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpyfpwne2c/_proc/Mkdocs.ipynb', '/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpyfpwne2c/_proc/index.ipynb', '/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpyfpwne2c/_proc/blogs/_blogs_index.ipynb', '/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpyfpwne2c/_proc/blogs/blogs_index.ipynb', '/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpyfpwne2c/_proc/blogs/qmd_blogs.qmd', '/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpyfpwne2c/_proc/guides/guides_index.ipynb', '/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpyfpwne2c/_proc/guides/_guides_index.ipynb', '/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn

Output created: _docs/README.md



In [ ]:
# | export


def _update_conditional_content_tags(text: str) -> str:
    """Update conditional content tags.

    Args:
        text: The text to update the conditional content tags in.

    Returns:
        The updated text with the conditional content tags modified.

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
    """
    pattern = r":::\s*{(?:\s*.*\.content-visible|\s*\.content-hidden)\s*(when|unless)-format\s*=\\\s*(\"|\')\s*(html|markdown)\s*\\(\"|\')\s*.*}"
    text = re.sub(
        pattern,
        lambda m: m.group(0).replace(
            m.group(1), "when" if m.group(1) == "unless" else "unless"
        ),
        text,
    )
    return text

In [ ]:
_input = """
may have some text before ::: {markdown=1 .content-visible when-format=\\"markdown\\" style=\\"text-align: center\\"}

some random text 

::: {.content-visible when-format=\\"markdown\\"}
"""
expected = """
may have some text before ::: {markdown=1 .content-visible unless-format=\\"markdown\\" style=\\"text-align: center\\"}

some random text 

::: {.content-visible unless-format=\\"markdown\\"}
"""

actual = _update_conditional_content_tags(_input)
print(actual)
assert actual == expected, actual

_input = ':::  { .content-visible when-format=\\"html\\"}'
expected = ':::  { .content-visible unless-format=\\"html\\"}'

actual = _update_conditional_content_tags(_input)
print(actual)
assert actual == expected, actual

_input = ':::  { .content-visible  unless-format=\\"html\\"}'
expected = ':::  { .content-visible  when-format=\\"html\\"}'

actual = _update_conditional_content_tags(_input)
print(actual)
assert actual == expected, actual

_input = ":::  { .content-visible  unless-format=\\'markdown\\'}"
expected = ":::  { .content-visible  when-format=\\'markdown\\'}"

actual = _update_conditional_content_tags(_input)
print(actual)
assert actual == expected, actual


may have some text before ::: {markdown=1 .content-visible unless-format=\"markdown\" style=\"text-align: center\"}

some random text 

::: {.content-visible unless-format=\"markdown\"}

:::  { .content-visible unless-format=\"html\"}
:::  { .content-visible  when-format=\"html\"}
:::  { .content-visible  when-format=\'markdown\'}


In [ ]:
_input = ":::  { .content-hidden  when-format=\\'html\\'}"
expected = ":::  { .content-hidden  unless-format=\\'html\\'}"

actual = _update_conditional_content_tags(_input)
print(actual)
assert actual == expected, actual

_input = '::: { .content-hidden  when-format=\\"markdown\\" }'
expected = '::: { .content-hidden  unless-format=\\"markdown\\" }'

actual = _update_conditional_content_tags(_input)
print(actual)
assert actual == expected, actual

_input = '::: { .content-hidden  unless-format=\\"html\\" }'
expected = '::: { .content-hidden  when-format=\\"html\\" }'

actual = _update_conditional_content_tags(_input)
print(actual)
assert actual == expected, actual

_input = '::: { .content-hidden  unless-format=\\"markdown\\" }'
expected = '::: { .content-hidden  when-format=\\"markdown\\" }'

actual = _update_conditional_content_tags(_input)
print(actual)
assert actual == expected, actual

:::  { .content-hidden  unless-format=\'html\'}
::: { .content-hidden  unless-format=\"markdown\" }
::: { .content-hidden  when-format=\"html\" }
::: { .content-hidden  when-format=\"markdown\" }


In [ ]:
# | export


def _update_mermaid_chart_tags(text: str) -> str:
    """Convert the mermaid chart tags from quarto format to markdown format.

    Args:
        text: The text to update the mermaid chart tags in.

    Returns:
        The updated text with the mermaid chart tags modified.

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
    """
    pattern = r"```\s*{mermaid\s*}"
    text = re.sub(pattern, "``` mermaid", text)
    return text

In [ ]:
valid_inputs = [
    "\n```{mermaid}\nflowchart LR\n ",
    "\n\n```   {mermaid}   \nflowchart LR\n ",
]
expected = ["\n``` mermaid\nflowchart LR\n ", "\n\n``` mermaid   \nflowchart LR\n "]

for n, i in enumerate(valid_inputs):
    actual = _update_mermaid_chart_tags(i)
    print(actual)
    assert actual == expected[n], actual


invalid_inputs = [
    "\n```{ mermaid}\nflowchart LR\n ",
    "\n\n```  some text {mermaid}   \nflowchart LR\n ",
]
expected = [
    "\n```{ mermaid}\nflowchart LR\n ",
    "\n\n```  some text {mermaid}   \nflowchart LR\n ",
]

for n, i in enumerate(invalid_inputs):
    actual = _update_mermaid_chart_tags(i)
    print(actual)
    assert actual == expected[n], actual


``` mermaid
flowchart LR
 


``` mermaid   
flowchart LR
 

```{ mermaid}
flowchart LR
 


```  some text {mermaid}   
flowchart LR
 


In [ ]:
# | export


def _add_markdown_attribute_to_enable_md_in_html(text: str) -> str:
    """Add markdown attribute to enable markdown in html.

    Args:
        text: The text to add the markdown attribute to

    Returns:
        The text with the markdown attribute added

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
    """
    pattern = r":::\s*{\s*(markdown=1)?\s*"
    text = re.sub(pattern, r"::: {markdown=1 ", text)
    return text

In [ ]:
_input = """
Sample markdown Content :::  {.content-visible when-format="html"}

This is a *Markdown* Paragraph.

:::
Sample markdown Content ::: {markdown=1 .content-visible when-format="html"}
This is a *Markdown* Paragraph.

:::
"""
expected = """
Sample markdown Content ::: {markdown=1 .content-visible when-format="html"}

This is a *Markdown* Paragraph.

:::
Sample markdown Content ::: {markdown=1 .content-visible when-format="html"}
This is a *Markdown* Paragraph.

:::
"""
actual = _add_markdown_attribute_to_enable_md_in_html(_input)
print(actual)

assert actual == expected


Sample markdown Content ::: {markdown=1 .content-visible when-format="html"}

This is a *Markdown* Paragraph.

:::
Sample markdown Content ::: {markdown=1 .content-visible when-format="html"}
This is a *Markdown* Paragraph.

:::



In [ ]:
# | export


def _update_quarto_tags_to_markdown_format(nb_path: Path) -> None:
    """Update Quarto tags to Markdown format

    Args:
        nb_path: Path to the notebook

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
    """
    with open(nb_path, "r") as f:
        contents = f.read()

    contents = _update_conditional_content_tags(contents)
    contents = _update_mermaid_chart_tags(contents)
    contents = _add_markdown_attribute_to_enable_md_in_html(contents)

    with open(nb_path, "w") as f:
        f.write(contents)

In [ ]:
def _create_test_notebook_contents() -> Dict[str, Any]:
    """Create a sample notebook using json"""
    return {
        "cells": [
            {
                "cell_type": "markdown",
                "metadata": {},
                "source": '::: {.content-visible when-format="markdown"}\n\nWill only appear in markdown.\n\n:::\n\n```{mermaid}\nflowchart LR\n  A[Hard edge] --> B(Round edge)\n  B --> C{Decision}\n  C --> D[Result one]\n  C --> E[Result two]\n```\n',
            },
            {
                "cell_type": "markdown",
                "metadata": {},
                "source": '::: {.content-visible when-format="html"}\n\nWill only appear in html.\n\n:::',
            },
        ],
        "metadata": {
            "kernelspec": {
                "display_name": "Python 3",
                "language": "python",
                "name": "python3",
            },
            "language_info": {
                "codemirror_mode": {"name": "ipython", "version": 3},
                "file_extension": ".py",
                "mimetype": "text/x-python",
                "name": "python",
                "nbconvert_exporter": "python",
                "pygments_lexer": "ipython3",
                "version": "3.8.3",
            },
        },
        "nbformat": 4,
        "nbformat_minor": 4,
    }

In [ ]:
test_nb_contents = _create_test_notebook_contents()

with TemporaryDirectory() as d:
    run_nbdev_new(d)
    fname = Path(d) / "test.ipynb"
    with open(fname, "w") as f:
        f.write(json.dumps(test_nb_contents))

    _update_quarto_tags_to_markdown_format(fname)

    with open(fname, "r") as f:
        contents = json.loads(f.read())


print(contents)

assert (
    '{markdown=1 .content-visible unless-format="markdown"}'
    in contents["cells"][0]["source"]
)
assert (
    not '{markdown=1 .content-visible when-format="markdown"}'
    in contents["cells"][0]["source"]
)

assert "mermaid" in contents["cells"][0]["source"]
assert not "{mermaid}" in contents["cells"][0]["source"]

assert (
    '{markdown=1 .content-visible unless-format="html"}'
    in contents["cells"][1]["source"]
)
assert (
    not '{markdown=1 .content-visible when-format="html"}'
    in contents["cells"][1]["source"]
)

settings.ini created.


pandoc -o README.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  output-file: index.html
  standalone: true
  default-image-extension: png
  
metadata
  title: Material for nbdev
  


{'cells': [{'cell_type': 'markdown', 'metadata': {}, 'source': '::: {markdown=1 .content-visible unless-format="markdown"}\n\nWill only appear in markdown.\n\n:::\n\n``` mermaid\nflowchart LR\n  A[Hard edge] --> B(Round edge)\n  B --> C{Decision}\n  C --> D[Result one]\n  C --> E[Result two]\n```\n'}, {'cell_type': 'markdown', 'metadata': {}, 'source': '::: {markdown=1 .content-visible unless-format="html"}\n\nWill only appear in html.\n\n:::'}], 'metadata': {'kernelspec': {'display_name': 'Python 3', 'language': 'python', 'name': 'python3'}, 'language_info': {'codemirror_mode': {'name': 'ipython', 'version': 3}, 'file_extension': '.py', 'mimetype': 'text/x-python', 'name': 'python', 'nbconvert_exporter': 'python', 'pygments_lexer': 'ipython3', 'version': '3.8.3'}}, 'nbformat': 4, 'nbformat_minor': 4}


Output created: _docs/README.md



In [ ]:
# | export


def _sprun(cmd: str) -> None:
    """Run a command via subprocess.check_output

    Args:
        cmd: The command to run

    Raises:
        subprocess.CalledProcessError: If the command fails

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
    """
    try:
        # nosemgrep: python.lang.security.audit.subprocess-shell-true.subprocess-shell-true
        subprocess.check_output(
            cmd, shell=True  # nosec: B602:subprocess_popen_with_shell_equals_true
        )

    except subprocess.CalledProcessError as e:
        sys.exit(
            f"CMD Failed: e={e}\n e.returncode={e.returncode}\n e.output={e.output}\n e.stderr={e.stderr}\n cmd={cmd}"
        )

In [ ]:
# | export


def _generate_markdown_from_files(root_path: str) -> None:
    """Generate markdown files from notebook files.

    Args:
        root_path: The root path of the project.

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
    """
    doc_path = Path(root_path) / "mkdocs" / "docs"
    doc_path.mkdir(exist_ok=True, parents=True)

    with set_cwd(root_path):

        cache = proc_nbs()
        files = _get_files_to_convert_to_markdown(cache)

        for f in files:
            dir_prefix = str(f.parent)[len(str(cache)) + 1 :]
            dst_md = doc_path / f"{dir_prefix}" / f"{f.stem}.md"
            dst_md.parent.mkdir(parents=True, exist_ok=True)

            _update_quarto_tags_to_markdown_format(f)

            cmd = f'cd "{cache}" && quarto render "{f}" -o "{f.stem}.md" -t gfm --no-execute'
            _sprun(cmd)

            src_md = cache / "_docs" / f"{f.stem}.md"
            shutil.move(src_md, dst_md)

In [ ]:
with TemporaryDirectory() as d:
    run_nbdev_new(d)

    (Path(d) / "nbs" / "guides").mkdir(exist_ok=True)
    (Path(d) / "nbs" / "blogs").mkdir(exist_ok=True)

    _nbs_path = (
        Path(".") / "nbs" / "Mkdocs.ipynb"
        if Path("settings.ini").exists()
        else Path("..") / "nbs" / "Mkdocs.ipynb"
    )
    shutil.copyfile(_nbs_path, Path(d) / "nbs" / "Mkdocs.ipynb")

    create_sample_qmd_file(d)

    for i in ["guides", "blogs"]:
        shutil.copyfile(
            Path(d) / "nbs" / "index.ipynb",
            Path(d) / "nbs" / f"{i}" / f"{i}_index.ipynb",
        )
        shutil.copyfile(
            Path(d) / "nbs" / "sample.qmd",
            Path(d) / "nbs" / f"{i}" / f"qmd_{i}.qmd",
        )

    test_nbs = Path(d) / "nbs" / "test.ipynb"
    test_nb_contents = _create_test_notebook_contents()
    with open(test_nbs, "w") as f:
        f.write(json.dumps(test_nb_contents))

    new(d)

    _generate_markdown_from_files(d)

    # check markdown files
    print("Checks:")
    mds = list((Path(d) / "mkdocs" / "docs").glob("**/*.md"))
    mds = [str(md) for md in mds]
    print("\n".join(mds))
    assert len(mds) > 0, len(mds)

    for i in [
        "index.md",
        "sample.md",
        "guides/guides_index.md",
        "guides/qmd_guides.md",
        "blogs/blogs_index.md",
        "blogs/qmd_blogs.md",
        "Mkdocs.md",
    ]:
        assert f"{d}/mkdocs/docs/{i}" in mds, f"{d}/mkdocs/docs/{i}"

    with open(Path(d) / "_proc" / "test.ipynb", "r") as f:
        contents = json.loads(f.read())

print(contents)

assert '{markdown=1 .content-visible unless-format="markdown"}' in "".join(
    contents["cells"][0]["source"]
)
assert not '{markdown=1 .content-visible when-format="markdown"}' in "".join(
    contents["cells"][0]["source"]
)

assert '{markdown=1 .content-visible unless-format="html"}' in "".join(
    contents["cells"][2]["source"]
)
assert not '{markdown=1 .content-visible when-format="html"}' in "".join(
    contents["cells"][2]["source"]
)

settings.ini created.


pandoc -o README.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  output-file: index.html
  standalone: true
  default-image-extension: png
  
metadata
  title: Material for nbdev
  


Directory /private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp5dxbft93/mkdocs created.
File '/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp5dxbft93/mkdocs/mkdocs.yml' generated.
File '/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp5dxbft93/mkdocs/summary_template.txt' generated.


Output created: _docs/README.md

pandoc -o Mkdocs.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  standalone: true
  default-image-extension: png
  
Output created: _docs/Mkdocs.md

pandoc -o test.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  standalone: true
  default-image-extension: png
  
Output created: _docs/test.md

pandoc -o sample.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  standalone: true
  default-image-extension: png
  
metadata
  title: Sample
  author: Sample
  date: last-modified
  
Output created: _docs/sample.md

pandoc -o index.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  output-file: index.html
  standalone: true
  

Checks:
/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp5dxbft93/mkdocs/docs/sample.md
/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp5dxbft93/mkdocs/docs/Mkdocs.md
/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp5dxbft93/mkdocs/docs/index.md
/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp5dxbft93/mkdocs/docs/test.md
/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp5dxbft93/mkdocs/docs/blogs/qmd_blogs.md
/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp5dxbft93/mkdocs/docs/blogs/blogs_index.md
/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp5dxbft93/mkdocs/docs/guides/qmd_guides.md
/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp5dxbft93/mkdocs/docs/guides/guides_index.md
{'cells': [{'cell_type': 'markdown', 'metadata': {}, 'source': ['::: {markdown=1 .content-visible unless-format="markdown"}\n', '\n', 'Will only appear in markdown.\n', '\n', ':::\n', '\n', '``` mermaid\n', 'flowchart LR\n', '  A[Hard edge] --> B(Round edge)\n', '  B --> C{Decision}\n',

Output created: ../_docs/qmd_guides.md



In [ ]:
# | export


def _replace_all(text: str, dir_prefix: str) -> str:
    """Replace the images relative path in the markdown string

    Args:
        text: The markdown string
        dir_prefix: Sub directory prefix to append to the image's relative path

    Returns:
        The text with the updated images relative path

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
    """
    _replace = {}

    image_patterns = [
        (
            re.compile(r"!\[[^\]]*\]\(([^https?:\/\/].*?)\s*(\"(?:.*[^\"])\")?\s*\)"),
            "../images/nbs/",
        ),
        (
            re.compile(r"<img\s*src\s*=\s*\"([^http|https][^\"]*)\""),
            "../../images/nbs/",
        ),
    ]

    for pattern, image_path in image_patterns:
        matches = [match.groups()[0] for match in pattern.finditer(text)]
        if len(matches) > 0:
            for m in matches:
                _replace[m] = (
                    os.path.normpath(Path(image_path).joinpath(f"{dir_prefix}/{m}"))
                    if len(dir_prefix) > 0
                    else f"images/nbs/{m}"
                )

    for k, v in _replace.items():
        text = text.replace(k, v)

    return text

In [ ]:
text = """![Git Repo_Clone_Page](../img/test.png)
![Git Repo_Clone_Page](images/git_repo_clone_page.png)
![Test](https://github.com/airtai/nbdev-mkdocs/actions/workflows/test.yaml/badge.svg)
![](http://example.com/badge.svg)
![some test](https://www.test.com/styles/images/a.png)
![](https://test.com/photos/920382/pexels-photo-920382.jpeg?auto=compress&cs=tinysrgb&w=1600)
<img src="../images/circles.svg" />
<img src="https://documentation.divio.com/_images/overview.png" />
"""

expected = """![Git Repo_Clone_Page](../images/nbs/img/test.png)
![Git Repo_Clone_Page](../images/nbs/guides/images/git_repo_clone_page.png)
![Test](https://github.com/airtai/nbdev-mkdocs/actions/workflows/test.yaml/badge.svg)
![](http://example.com/badge.svg)
![some test](https://www.test.com/styles/images/a.png)
![](https://test.com/photos/920382/pexels-photo-920382.jpeg?auto=compress&cs=tinysrgb&w=1600)
<img src="../../images/nbs/images/circles.svg" />
<img src="https://documentation.divio.com/_images/overview.png" />
"""

dir_prefix = "guides"
actual = _replace_all(text, dir_prefix)
print(actual)
assert actual == expected, actual

text = """![Git Repo_Clone_Page](img/test.png)
![Git Repo_Clone_Page](guides/images/git_repo_clone_page.png)
![Test](https://github.com/airtai/nbdev-mkdocs/actions/workflows/test.yaml/badge.svg)
![](http://example.com/badge.svg)
![some test](https://www.test.com/styles/images/a.png)
![](https://test.com/photos/920382/pexels-photo-920382.jpeg?auto=compress&cs=tinysrgb&w=1600)
<img src="images/marie-curie-notebook.jpg" class="rounded preview-image"
alt="Photo of an opened research notebook with diagrams and writing in French" />
"""

expected = """![Git Repo_Clone_Page](images/nbs/img/test.png)
![Git Repo_Clone_Page](images/nbs/guides/images/git_repo_clone_page.png)
![Test](https://github.com/airtai/nbdev-mkdocs/actions/workflows/test.yaml/badge.svg)
![](http://example.com/badge.svg)
![some test](https://www.test.com/styles/images/a.png)
![](https://test.com/photos/920382/pexels-photo-920382.jpeg?auto=compress&cs=tinysrgb&w=1600)
<img src="images/nbs/images/marie-curie-notebook.jpg" class="rounded preview-image"
alt="Photo of an opened research notebook with diagrams and writing in French" />
"""

dir_prefix = ""
actual = _replace_all(text, dir_prefix)
print(actual)
assert actual == expected, actual

![Git Repo_Clone_Page](../images/nbs/img/test.png)
![Git Repo_Clone_Page](../images/nbs/guides/images/git_repo_clone_page.png)
![Test](https://github.com/airtai/nbdev-mkdocs/actions/workflows/test.yaml/badge.svg)
![](http://example.com/badge.svg)
![some test](https://www.test.com/styles/images/a.png)
![](https://test.com/photos/920382/pexels-photo-920382.jpeg?auto=compress&cs=tinysrgb&w=1600)
<img src="../../images/nbs/images/circles.svg" />
<img src="https://documentation.divio.com/_images/overview.png" />

![Git Repo_Clone_Page](images/nbs/img/test.png)
![Git Repo_Clone_Page](images/nbs/guides/images/git_repo_clone_page.png)
![Test](https://github.com/airtai/nbdev-mkdocs/actions/workflows/test.yaml/badge.svg)
![](http://example.com/badge.svg)
![some test](https://www.test.com/styles/images/a.png)
![](https://test.com/photos/920382/pexels-photo-920382.jpeg?auto=compress&cs=tinysrgb&w=1600)
<img src="images/nbs/images/marie-curie-notebook.jpg" class="rounded preview-image"
alt="Photo o

In [ ]:
# | export


def _update_path_in_markdown(cache: Path, doc_path: Path) -> None:
    """Update guide images relative path in the markdown files

    Args:
        cache: Path to the nbs cache directory
        doc_path: Path to the mkdocs/docs directory

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
    """
    files = _get_files_to_convert_to_markdown(cache)

    for file in files:
        dir_prefix = str(file.parent)[len(str(cache)) + 1 :]
        md = doc_path / f"{dir_prefix}" / f"{file.stem}.md"

        with open(Path(md), "r") as f:
            _new_text = f.read()
            _new_text = _replace_all(_new_text, dir_prefix)
        with open(Path(md), "w") as f:
            f.write(_new_text)


def _copy_images_to_docs_dir(root_path: str) -> None:
    # Reference: https://github.com/quarto-dev/quarto-cli/blob/main/src/core/image.ts#L38
    """Copy images from nbs to docs directory.

    Args:
        root_path: The root path of the project.

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
    """
    image_extensions = [
        ".apng",
        ".avif",
        ".gif",
        ".jpg",
        ".jpeg",
        ".jfif",
        ".pjpeg",
        ".pjp",
        ".png",
        ".svg",
        ".webp",
    ]

    cache = proc_nbs()
    nbs_images_path = [
        p for p in Path(cache).glob(r"**/*") if p.suffix in image_extensions
    ]

    if len(nbs_images_path) > 0:
        doc_path = Path(root_path) / "mkdocs" / "docs"
        img_path = Path(doc_path) / "images" / "nbs"
        for src_path in nbs_images_path:
            dir_prefix = str(src_path.parent)[len(str(cache)) + 1 :]
            dst_path = Path(img_path) / f"{dir_prefix}"
            dst_path.mkdir(exist_ok=True, parents=True)
            shutil.copy(src_path, dst_path)

        _update_path_in_markdown(cache, doc_path)

In [ ]:
def copy_guides(src, dst):
    """Copy guides from one directory to another.

    Args:
        src: Source directory
        dst: Destination directory

    Raises:
        AssertionError: If src does not exist

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
    """
    src = Path(src)
    dst = Path(dst)
    assert src.exists()
    src_guides = src / "nbs" / "guides"
    src_guides_len = len(src_guides.parts)

    dst_guides = dst / "nbs" / "guides"

    for ext in [".ipynb", ".png", ".jpeg", ".jpg"]:
        for src_f in src_guides.glob(f"**/*{ext}"):
            dst_parts = dst_guides.parts + src_f.parts[src_guides_len:]
            dst_f = Path(*dst_parts)
            dst_f.parent.mkdir(exist_ok=True, parents=True)

            print(f"{src_f}, {dst_f}")
            shutil.copyfile(src_f, dst_f)

In [ ]:
with TemporaryDirectory() as d:

    run_nbdev_new(d)

    settings_path = Path(d) / "settings.ini"
    assert settings_path.exists()

    _root_path = Path(".") if Path("settings.ini").exists() else Path("..")
    copy_guides(_root_path, d)
    shutil.copytree((Path(d) / "nbs" / "guides"), (Path(d) / "nbs" / "blogs"))

    for f in ["Mkdocs.ipynb", "Social_Image_Generator.ipynb"]:
        shutil.copyfile(_root_path / "nbs" / f, Path(d) / "nbs" / f)

    new(d)

    _generate_markdown_from_files(root_path=d)
    _copy_images_to_docs_dir(d)

    # check image files
    print("Checks for images:")
    imgs = list((Path(d) / "mkdocs" / "docs" / "images" / "nbs").glob("**/*.*"))
    print("\n".join([str(img) for img in imgs]))
    assert len(imgs) > 0, len(imgs)

    # check markdown files
    print("Checks for markdown:")
    mds = list((Path(d) / "mkdocs" / "docs").glob("**/*.md"))
    print("\n".join([str(md) for md in mds]))
    assert (Path(d) / "mkdocs" / "docs" / "index.md").exists()
    assert (Path(d) / "mkdocs" / "docs" / "Mkdocs.md").exists()
    assert (Path(d) / "mkdocs" / "docs" / "Social_Image_Generator.md").exists()
    assert (
        Path(d) / "mkdocs" / "docs" / "guides" / "Basic_User_Guide.md"
    ).exists()
    assert (
        Path(d) / "mkdocs" / "docs" / "blogs" / "Basic_User_Guide.md"
    ).exists()

    with open(
        (Path(d) / "mkdocs" / "docs" / "blogs" / "Basic_User_Guide.md"),
        "r",
    ) as f:
        contents = f.read()
        assert (
            "![Empty Git Repo](../images/nbs/blogs/images/empty_git_repo.png)"
            in contents
        )
        assert (
            "![Git Repo_Clone_Page](../images/nbs/blogs/images/git_repo_clone_page.png)"
            in contents
        )
        assert "![](../images/nbs/blogs/images/jupyter_home.png)" in contents

    (
        Path(d)
        / "mkdocs"
        / "docs"
        / "images"
        / "nbs"
        / "blogs"
        / "images"
        / "jupyter_home.png"
    ).exists()
    (
        Path(d)
        / "mkdocs"
        / "docs"
        / "images"
        / "nbs"
        / "guides"
        / "images"
        / "empty_git_repo.png"
    ).exists()

    print("OK.")

settings.ini created.


pandoc -o README.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  output-file: index.html
  standalone: true
  default-image-extension: png
  
metadata
  title: Material for nbdev
  


../nbs/guides/Material_for_MkDocs_Customization.ipynb, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpl1a9o4n_/nbs/guides/Material_for_MkDocs_Customization.ipynb
../nbs/guides/Customizing_The_Sidebar.ipynb, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpl1a9o4n_/nbs/guides/Customizing_The_Sidebar.ipynb
../nbs/guides/Guide_01_End_To_End_Walkthrough.ipynb, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpl1a9o4n_/nbs/guides/Guide_01_End_To_End_Walkthrough.ipynb
../nbs/guides/Add_Guides.ipynb, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpl1a9o4n_/nbs/guides/Add_Guides.ipynb
../nbs/guides/Basic_User_Guide.ipynb, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpl1a9o4n_/nbs/guides/Basic_User_Guide.ipynb
../nbs/guides/Add_Release_Notes.ipynb, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpl1a9o4n_/nbs/guides/Add_Release_Notes.ipynb
../nbs/guides/Configure_Social_Share_Image.ipynb, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpl1a9o4n_/nbs/guides/Configure_So

Output created: _docs/README.md

pandoc -o Mkdocs.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  standalone: true
  default-image-extension: png
  
Output created: _docs/Mkdocs.md

pandoc -o index.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  output-file: index.html
  standalone: true
  default-image-extension: png
  
metadata
  title: Material for nbdev
  
Output created: _docs/index.md

pandoc -o Social_Image_Generator.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  standalone: true
  default-image-extension: png
  
Output created: _docs/Social_Image_Generator.md

pandoc -o ../Material_for_MkDocs_Customization.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+

Checks for images:
/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpl1a9o4n_/mkdocs/docs/images/nbs/blogs/images/say_hello.png
/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpl1a9o4n_/mkdocs/docs/images/nbs/blogs/images/docstring-gen-extension-btn.png
/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpl1a9o4n_/mkdocs/docs/images/nbs/blogs/images/git_repo_clone_page.png
/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpl1a9o4n_/mkdocs/docs/images/nbs/blogs/images/set_github_token_permission.png
/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpl1a9o4n_/mkdocs/docs/images/nbs/blogs/images/sidebar_1.png
/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpl1a9o4n_/mkdocs/docs/images/nbs/blogs/images/CLI_command.png
/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpl1a9o4n_/mkdocs/docs/images/nbs/blogs/images/sidebar_2.png
/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpl1a9o4n_/mkdocs/docs/images/nbs/blogs/images/foo_doc_string.png
/var/folders/6n/3rjds7v52cd83wqkd565db0h

### Build summary for guides

In [ ]:
# | export


def _get_title_from_notebook(file_path: Path) -> str:
    """Get the title of a notebook or markdown file.

    Args:
        file_path: The path to the file.

    Returns:
        The title of the file.

    Raises:
        ValueError: If the file does not exist.

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
    """
    title: str
    cache = proc_nbs()
    _file_path = Path(cache) / file_path

    if not _file_path.exists():
        typer.secho(
            f"Unexpected error: path {_file_path.resolve()} does not exists!",
            err=True,
            fg=typer.colors.RED,
        )
        raise typer.Exit(code=1)

    if _file_path.suffix == ".ipynb":
        nbp = NBProcessor(_file_path, procs=FrontmatterProc)
        nbp.process()

        if "title" in nbp.nb.frontmatter_:
            title = nbp.nb.frontmatter_["title"]
        else:
            headers = [
                cell["source"]
                for cell in nbp.nb["cells"]
                if cell["cell_type"] == "markdown" and cell["source"].startswith("#")
            ]
            title = (
                f"{_file_path.stem}.html"
                if len(headers) == 0
                else headers[0].replace("#", "").strip()
            )
    else:
        with open(_file_path) as f:
            contents = f.read()
        metadata = _fm2dict(contents, nb=False)
        metadata = {k.lower(): v for k, v in metadata.items()}
        title = metadata["title"]

    return title

In [ ]:
with TemporaryDirectory() as d:
    run_nbdev_new(d)

    settings_path = Path(d) / "settings.ini"
    assert settings_path.exists()

    _root_path = Path(".") if Path("settings.ini").exists() else Path("..")
    copy_guides(_root_path, d)

    for f in ["Mkdocs.ipynb", "Social_Image_Generator.ipynb"]:
        shutil.copyfile(_root_path / "nbs" / f, Path(d) / "nbs" / f)

    create_sample_qmd_file(d)

    new(d)

    cmd = f'cd "{d}" && nbdev_docs'
    _sprun(cmd)

    _generate_markdown_from_files(d)

    nb_paths = [
        Path("index.ipynb"),
        Path("sample.qmd"),
        Path("Mkdocs.ipynb"),
        Path("Social_Image_Generator.ipynb"),
        Path("guides/Basic_User_Guide.ipynb"),
    ]

    expected = [
        "Material for nbdev",
        "Sample",
        "Create new",
        "Social_Image_Generator.html",
        "Basic User Guide",
    ]
    actual = []
    for nb_path in nb_paths:
        actual.append(_get_title_from_notebook(nb_path))

    print(actual)

    assert actual == expected

settings.ini created.


pandoc -o README.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  output-file: index.html
  standalone: true
  default-image-extension: png
  
metadata
  title: Material for nbdev
  


../nbs/guides/Material_for_MkDocs_Customization.ipynb, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpyqle7tzh/nbs/guides/Material_for_MkDocs_Customization.ipynb
../nbs/guides/Customizing_The_Sidebar.ipynb, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpyqle7tzh/nbs/guides/Customizing_The_Sidebar.ipynb
../nbs/guides/Guide_01_End_To_End_Walkthrough.ipynb, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpyqle7tzh/nbs/guides/Guide_01_End_To_End_Walkthrough.ipynb
../nbs/guides/Add_Guides.ipynb, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpyqle7tzh/nbs/guides/Add_Guides.ipynb
../nbs/guides/Basic_User_Guide.ipynb, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpyqle7tzh/nbs/guides/Basic_User_Guide.ipynb
../nbs/guides/Add_Release_Notes.ipynb, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpyqle7tzh/nbs/guides/Add_Release_Notes.ipynb
../nbs/guides/Configure_Social_Share_Image.ipynb, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpyqle7tzh/nbs/guides/Configure_So

Output created: _docs/README.md

[ 1/12] Mkdocs.ipynb
[ 2/12] guides/Material_for_MkDocs_Customization.ipynb
[ 3/12] guides/Customizing_The_Sidebar.ipynb
[ 4/12] guides/Guide_01_End_To_End_Walkthrough.ipynb
[ 5/12] guides/Add_Guides.ipynb
[ 6/12] guides/Basic_User_Guide.ipynb
[ 7/12] guides/Add_Release_Notes.ipynb
[ 8/12] guides/Configure_Social_Share_Image.ipynb
[ 9/12] guides/Auto_Generate_Docstrings.ipynb
[10/12] sample.qmd
[11/12] index.ipynb
[12/12] Social_Image_Generator.ipynb

Output created: _docs/index.html

pandoc -o Mkdocs.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  standalone: true
  default-image-extension: png
  
Output created: _docs/Mkdocs.md

pandoc -o sample.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  standalone: true
  default-image-extension: png
  
metadata
  title: Sample
  author: Sam

['Material for nbdev', 'Sample', 'Create new', 'Social_Image_Generator.html', 'Basic User Guide']


In [ ]:
# | export


def _get_sidebar_from_config(file_path: Path) -> List[Any]:
    """Get the sidebar contents from the sidebar.yml or _quarto.yml file.

    Args:
        file_path: Path to the sidebar.yml or _quarto.yml file.

    Returns:
        The sidebar contents.

    Raises:
        KeyError: If the sidebar is not defined in the config file.

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
    """
    if not file_path.exists():
        typer.secho(
            f"Path '{file_path.resolve()}' does not exists!",
            err=True,
            fg=typer.colors.RED,
        )
        raise typer.Exit(code=1)

    try:
        with open(file_path) as f:
            config = yaml.safe_load(f)
        sidebar: List[Any] = config["website"]["sidebar"]["contents"]
    except KeyError as e:
        typer.secho(
            f"Key Error: Contents of the sidebar are not defined in the files sidebar.yml or _quarto.yml.",
            err=True,
            fg=typer.colors.RED,
        )
        raise typer.Exit(code=1)

    return sidebar


def _read_sidebar_from_yml(root_path: str) -> List[Union[str, Any]]:
    """Get the sidebar contents from the sidebar.yml or _quarto.yml file.

    Args:
        root_path: The root path of the project.

    Returns:
        A list of strings and objects.

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
    """
    _proc_dir = Path(root_path) / "_proc"
    sidebar_yml_path = _proc_dir / "sidebar.yml"
    _quarto_yml_path = _proc_dir / "_quarto.yml"

    custom_sidebar = get_value_from_config(root_path, "custom_sidebar")
    if custom_sidebar == "False":
        cmd = f'cd "{root_path}" && nbdev_docs'
        _sprun(cmd)

    return (
        _get_sidebar_from_config(sidebar_yml_path)
        if sidebar_yml_path.exists()
        else _get_sidebar_from_config(_quarto_yml_path)
    )

In [ ]:
with TemporaryDirectory() as d:
    run_nbdev_new(d)
    create_sample_qmd_file(d)
    new(d)

    with set_cwd(d):
        sidebar = _read_sidebar_from_yml(d)
    print(sidebar)
    assert sidebar == ["index.ipynb", "sample.qmd"]

settings.ini created.


pandoc -o README.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  output-file: index.html
  standalone: true
  default-image-extension: png
  
metadata
  title: Material for nbdev
  


Directory /private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpsynqps0j/mkdocs created.
File '/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpsynqps0j/mkdocs/mkdocs.yml' generated.
File '/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpsynqps0j/mkdocs/summary_template.txt' generated.


Output created: _docs/README.md

[1/2] sample.qmd
[2/2] index.ipynb


['index.ipynb', 'sample.qmd']



Output created: _docs/index.html



In [ ]:
with pytest.raises(typer.Exit) as e:
    with TemporaryDirectory() as d:
        run_nbdev_new(d)

        new(d)

        updater = ConfigUpdater()
        updater.read(Path(d) / "settings.ini")
        updater["DEFAULT"]["custom_sidebar"] = True
        updater.update_file()
        with set_cwd(d):
            _read_sidebar_from_yml(d)

print("OK")

settings.ini created.


pandoc -o README.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  output-file: index.html
  standalone: true
  default-image-extension: png
  
metadata
  title: Material for nbdev
  


Directory /private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpttavjl6s/mkdocs created.
File '/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpttavjl6s/mkdocs/mkdocs.yml' generated.
File '/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpttavjl6s/mkdocs/summary_template.txt' generated.


Output created: _docs/README.md

Key Error: Contents of the sidebar are not defined in the files sidebar.yml or _quarto.yml.


OK


In [ ]:
# | export


def _flattern_sidebar_items(items: List[Union[str, Any]]) -> List[Union[str, Any]]:
    """Flatten a list of items.

    Args:
        items: A list of items.

    Returns:
        A flattened list of items.

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
    """
    return [i for item in items if isinstance(item, list) for i in item] + [
        item for item in items if not isinstance(item, list)
    ]


def _expand_sidebar_if_needed(
    root_path: str, sidebar: List[Union[str, Any]]
) -> List[Union[str, Any]]:
    """Expand the sidebar if needed.

    Args:
        root_path: The root path of the project
        sidebar: The sidebar to expand

    Returns:
        The expanded sidebar

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
    """
    _proc_dir = Path(root_path) / "_proc"
    exts = [".ipynb", ".qmd"]

    for index, item in enumerate(sidebar):
        if "auto" in item:
            files = list(_proc_dir.glob("".join(item["auto"].split("/")[1:])))  # type: ignore
            files = sorted([str(f.relative_to(_proc_dir)) for f in files if f.suffix in exts])  # type: ignore
            sidebar[index] = files

        if isinstance(item, dict) and "contents" in item:
            _contents = item["contents"]
            if isinstance(_contents, str) and bool(re.search(r"[*?\[\]]", _contents)):
                files = list(_proc_dir.glob(item["contents"]))
                files = sorted([str(f.relative_to(_proc_dir)) for f in files if f.suffix in exts])  # type: ignore
                item["contents"] = files

    flat_sidebar = _flattern_sidebar_items(sidebar)
    return flat_sidebar

In [ ]:
_sidebar = """- auto: \"/*.ipynb\"
- section: Blogs
  contents: blogs/*
- section: Guides
  contents: guides/*
- section: Explanations
  contents:
  - explanations/explanation_1.ipynb
  - explanations/explanation_2.ipynb
"""

expected = [
    "getting_started.ipynb",
    "index.ipynb",
    {"section": "Blogs", "contents": ["blogs/blog_1.ipynb", "blogs/blog_2.ipynb"]},
    {"section": "Guides", "contents": ['guides/Add_Guides.ipynb', 'guides/Add_Release_Notes.ipynb', 'guides/Auto_Generate_Docstrings.ipynb', 'guides/Basic_User_Guide.ipynb', 'guides/Configure_Social_Share_Image.ipynb', 'guides/Customizing_The_Sidebar.ipynb', 'guides/Material_for_MkDocs_Customization.ipynb']},
    {
        "section": "Explanations",
        "contents": [
            "explanations/explanation_1.ipynb",
            "explanations/explanation_2.ipynb",
        ],
    },
]


with TemporaryDirectory() as d:
    run_nbdev_new(d)
    _root_path = Path(".") if Path("settings.ini").exists() else Path("..")
    copy_guides(_root_path, d)

    shutil.copyfile(
        Path(d) / "nbs" / "index.ipynb", Path(d) / "nbs" / "getting_started.ipynb"
    )

    (Path(d) / "nbs" / "blogs").mkdir(parents=True)
    for i in ["blog_1", "blog_2"]:
        shutil.copyfile(
            Path(d) / "nbs" / "index.ipynb", (Path(d) / "nbs" / "blogs" / f"{i}.ipynb")
        )

    (Path(d) / "nbs" / "explanations").mkdir(parents=True)
    for i in ["explanation_1", "explanation_2"]:
        shutil.copyfile(
            Path(d) / "nbs" / "index.ipynb",
            (Path(d) / "nbs" / "explanations" / f"{i}.ipynb"),
        )

    new(d)

    # update _quarto.yml file
    _quarto_yml_path = Path(d) / "nbs" / "_quarto.yml"
    _yaml = YAML()
    config = _yaml.load(_quarto_yml_path)
    config["website"]["sidebar"]["contents"] = _yaml.load(_sidebar)
    _yaml.dump(config, _quarto_yml_path)

    # Set the custom_sidebar flag to True
    updater = ConfigUpdater()
    updater.read(Path(d) / "settings.ini")
    updater["DEFAULT"]["custom_sidebar"] = True
    updater.update_file()

    _generate_markdown_from_files(d)
    _copy_images_to_docs_dir(d)

    sidebar = _read_sidebar_from_yml(d)

    actual = _expand_sidebar_if_needed(d, sidebar)

    print(actual)
    assert actual == expected, actual

settings.ini created.


pandoc -o README.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  output-file: index.html
  standalone: true
  default-image-extension: png
  
metadata
  title: Material for nbdev
  


../nbs/guides/Material_for_MkDocs_Customization.ipynb, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmprfx_z6pd/nbs/guides/Material_for_MkDocs_Customization.ipynb
../nbs/guides/Customizing_The_Sidebar.ipynb, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmprfx_z6pd/nbs/guides/Customizing_The_Sidebar.ipynb
../nbs/guides/Add_Guides.ipynb, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmprfx_z6pd/nbs/guides/Add_Guides.ipynb
../nbs/guides/Basic_User_Guide.ipynb, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmprfx_z6pd/nbs/guides/Basic_User_Guide.ipynb
../nbs/guides/Add_Release_Notes.ipynb, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmprfx_z6pd/nbs/guides/Add_Release_Notes.ipynb
../nbs/guides/Configure_Social_Share_Image.ipynb, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmprfx_z6pd/nbs/guides/Configure_Social_Share_Image.ipynb
../nbs/guides/Auto_Generate_Docstrings.ipynb, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmprfx_z6pd/nbs/guides/Auto_Generate_Docstrin

Output created: _docs/README.md

pandoc -o getting_started.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  output-file: getting_started.html
  standalone: true
  default-image-extension: png
  
metadata
  title: Material for nbdev
  
Output created: _docs/getting_started.md

pandoc -o index.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  output-file: index.html
  standalone: true
  default-image-extension: png
  
metadata
  title: Material for nbdev
  
Output created: _docs/index.md

pandoc -o ../blog_1.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  output-file: blog_1.html
  standalone: true
  default-image-extension: png
  
metadata
  title: Material for nbdev
  
Output created: ../_docs/blog_1.md

pandoc -o ../blog_2.md
  to:

['getting_started.ipynb', 'index.ipynb', {'section': 'Blogs', 'contents': ['blogs/blog_1.ipynb', 'blogs/blog_2.ipynb']}, {'section': 'Guides', 'contents': ['guides/Add_Guides.ipynb', 'guides/Add_Release_Notes.ipynb', 'guides/Auto_Generate_Docstrings.ipynb', 'guides/Basic_User_Guide.ipynb', 'guides/Configure_Social_Share_Image.ipynb', 'guides/Customizing_The_Sidebar.ipynb', 'guides/Material_for_MkDocs_Customization.ipynb']}, {'section': 'Explanations', 'contents': ['explanations/explanation_1.ipynb', 'explanations/explanation_2.ipynb']}]


In [ ]:
# | export


def _generate_nav_from_sidebar(
    sidebar_items: List[Union[str, Dict[str, Any]]], level: int = 0
) -> str:
    """Generate a navigation string for mkdocs from a sidebar list.

    Args:
        sidebar_items: A list of strings or dictionaries.
        level: The level of indentation to use.

    Returns:
        str: The navigation string.

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
    """
    output = ""
    links = [
        "{}- [{}]({}.md)\n".format(
            "    " * level,
            _get_title_from_notebook(Path(item)),
            Path(item).with_suffix(""),
        )
        if isinstance(item, str)
        else "{}- {}\n".format("    " * level, item["section"])
        + _generate_nav_from_sidebar(item["contents"], level + 1)
        for item in sidebar_items
    ]
    output += "".join(links)
    return output

In [ ]:
with TemporaryDirectory() as d:
    run_nbdev_new(d)

    settings_path = Path(d) / "settings.ini"
    assert settings_path.exists()

    copy_guides(Path(".") if Path("settings.ini").exists() else Path(".."), d)

    _root_path = Path(".") if Path("settings.ini").exists() else Path("..")

    for f in ["Mkdocs.ipynb", "Social_Image_Generator.ipynb"]:
        shutil.copyfile(_root_path / "nbs" / f, Path(d) / "nbs" / f)

    (Path(d) / "nbs" / "api").mkdir(parents=True)
    shutil.copyfile(
        Path(d) / "nbs" / "Mkdocs.ipynb", (Path(d) / "nbs" / "api" / "Mkdocs.ipynb")
    )

    shutil.copytree((Path(d) / "nbs" / "guides"), (Path(d) / "nbs" / "blogs"))

    create_sample_qmd_file(d)

    new(d)
    with set_cwd(d):
        sidebar = _read_sidebar_from_yml(d)
        expanded_sidebar = _expand_sidebar_if_needed(d, sidebar)
        assert expanded_sidebar == [
            "index.ipynb",
            "Mkdocs.ipynb",
            "Social_Image_Generator.ipynb",
            "sample.qmd",
            {"section": "api", "contents": ["api/Mkdocs.ipynb"]},
            {
                "section": "blogs",
                "contents": ['blogs/Add_Guides.ipynb', 'blogs/Add_Release_Notes.ipynb', 'blogs/Auto_Generate_Docstrings.ipynb', 'blogs/Basic_User_Guide.ipynb', 'blogs/Configure_Social_Share_Image.ipynb', 'blogs/Customizing_The_Sidebar.ipynb', 'blogs/Material_for_MkDocs_Customization.ipynb'],
            },
            {
                "section": "guides",
                "contents": ['guides/Add_Guides.ipynb', 'guides/Add_Release_Notes.ipynb', 'guides/Auto_Generate_Docstrings.ipynb', 'guides/Basic_User_Guide.ipynb', 'guides/Configure_Social_Share_Image.ipynb', 'guides/Customizing_The_Sidebar.ipynb', 'guides/Material_for_MkDocs_Customization.ipynb'],
            },
        ]

        actual = _generate_nav_from_sidebar(expanded_sidebar)
        print(actual)

        expected = """- [Material for nbdev](index.md)
- [Create new](Mkdocs.md)
- [Social_Image_Generator.html](Social_Image_Generator.md)
- [Sample](sample.md)
- api
    - [Create new](api/Mkdocs.md)
- blogs
    - [Add guides](blogs/Add_Guides.md)
    - [Add release notes](blogs/Add_Release_Notes.md)
    - [Auto generate docstrings](blogs/Auto_Generate_Docstrings.md)
    - [Basic User Guide](blogs/Basic_User_Guide.md)
    - [Configure social share image](blogs/Configure_Social_Share_Image.md)
    - [Customizing the sidebar](blogs/Customizing_The_Sidebar.md)
    - [Material for MkDocs Customization](blogs/Material_for_MkDocs_Customization.md)
- guides
    - [Add guides](guides/Add_Guides.md)
    - [Add release notes](guides/Add_Release_Notes.md)
    - [Auto generate docstrings](guides/Auto_Generate_Docstrings.md)
    - [Basic User Guide](guides/Basic_User_Guide.md)
    - [Configure social share image](guides/Configure_Social_Share_Image.md)
    - [Customizing the sidebar](guides/Customizing_The_Sidebar.md)
    - [Material for MkDocs Customization](guides/Material_for_MkDocs_Customization.md)
"""

        assert actual == expected, actual

settings.ini created.


pandoc -o README.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  output-file: index.html
  standalone: true
  default-image-extension: png
  
metadata
  title: Material for nbdev
  


../nbs/guides/Material_for_MkDocs_Customization.ipynb, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp6nlinl2s/nbs/guides/Material_for_MkDocs_Customization.ipynb
../nbs/guides/Customizing_The_Sidebar.ipynb, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp6nlinl2s/nbs/guides/Customizing_The_Sidebar.ipynb
../nbs/guides/Add_Guides.ipynb, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp6nlinl2s/nbs/guides/Add_Guides.ipynb
../nbs/guides/Basic_User_Guide.ipynb, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp6nlinl2s/nbs/guides/Basic_User_Guide.ipynb
../nbs/guides/Add_Release_Notes.ipynb, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp6nlinl2s/nbs/guides/Add_Release_Notes.ipynb
../nbs/guides/Configure_Social_Share_Image.ipynb, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp6nlinl2s/nbs/guides/Configure_Social_Share_Image.ipynb
../nbs/guides/Auto_Generate_Docstrings.ipynb, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp6nlinl2s/nbs/guides/Auto_Generate_Docstrin

Output created: _docs/README.md

[ 1/19] blogs/Material_for_MkDocs_Customization.ipynb
[ 2/19] blogs/Customizing_The_Sidebar.ipynb
[ 3/19] blogs/Add_Guides.ipynb
[ 4/19] blogs/Basic_User_Guide.ipynb
[ 5/19] blogs/Add_Release_Notes.ipynb
[ 6/19] blogs/Configure_Social_Share_Image.ipynb
[ 7/19] blogs/Auto_Generate_Docstrings.ipynb
[ 8/19] Mkdocs.ipynb
[ 9/19] guides/Material_for_MkDocs_Customization.ipynb
[10/19] guides/Customizing_The_Sidebar.ipynb
[11/19] guides/Add_Guides.ipynb
[12/19] guides/Basic_User_Guide.ipynb
[13/19] guides/Add_Release_Notes.ipynb
[14/19] guides/Configure_Social_Share_Image.ipynb
[15/19] guides/Auto_Generate_Docstrings.ipynb
[16/19] api/Mkdocs.ipynb
[17/19] sample.qmd
[18/19] index.ipynb
[19/19] Social_Image_Generator.ipynb

Output created: _docs/index.html



- [Material for nbdev](index.md)
- [Create new](Mkdocs.md)
- [Social_Image_Generator.html](Social_Image_Generator.md)
- [Sample](sample.md)
- api
    - [Create new](api/Mkdocs.md)
- blogs
    - [Add guides](blogs/Add_Guides.md)
    - [Add release notes](blogs/Add_Release_Notes.md)
    - [Auto generate docstrings](blogs/Auto_Generate_Docstrings.md)
    - [Basic User Guide](blogs/Basic_User_Guide.md)
    - [Configure social share image](blogs/Configure_Social_Share_Image.md)
    - [Customizing the sidebar](blogs/Customizing_The_Sidebar.md)
    - [Material for MkDocs Customization](blogs/Material_for_MkDocs_Customization.md)
- guides
    - [Add guides](guides/Add_Guides.md)
    - [Add release notes](guides/Add_Release_Notes.md)
    - [Auto generate docstrings](guides/Auto_Generate_Docstrings.md)
    - [Basic User Guide](guides/Basic_User_Guide.md)
    - [Configure social share image](guides/Configure_Social_Share_Image.md)
    - [Customizing the sidebar](guides/Customizing_The_Sidebar.md)

In [ ]:
# | export


def _generate_summary_for_sidebar(
    root_path: str,
) -> str:
    """Generate a summary for the sidebar

    Args:
        root_path: The root path

    Returns:
        The summary for the sidebar

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
    """
    with set_cwd(root_path):
        sidebar = _read_sidebar_from_yml(root_path)
        expanded_sidebar = _expand_sidebar_if_needed(root_path, sidebar)
        sidebar_nav = _generate_nav_from_sidebar(expanded_sidebar)

        return sidebar_nav

In [ ]:
with TemporaryDirectory() as d:
    run_nbdev_new(d)

    settings_path = Path(d) / "settings.ini"
    # copy nbs folder
    _root_path = Path(".") if Path("settings.ini").exists() else Path("..")

    copy_guides(_root_path, d)
    for f in ["Mkdocs.ipynb", "Social_Image_Generator.ipynb"]:
        shutil.copyfile(_root_path / "nbs" / f, Path(d) / "nbs" / f)

    (Path(d) / "nbs" / "api").mkdir(parents=True)
    shutil.copyfile(
        Path(d) / "nbs" / "Mkdocs.ipynb", (Path(d) / "nbs" / "api" / "Mkdocs.ipynb")
    )

    shutil.copytree((Path(d) / "nbs" / "guides"), (Path(d) / "nbs" / "blogs"))

    cmd = f'cd "{d}" && nbdev_sidebar'
    print(f"executing the command: {cmd}")
    _sprun(cmd)

    new(d)

    actual = _generate_summary_for_sidebar(d)

    print(actual)

    expected = """- [Material for nbdev](index.md)
- [Create new](Mkdocs.md)
- [Social_Image_Generator.html](Social_Image_Generator.md)
- api
    - [Create new](api/Mkdocs.md)
- blogs
    - [Add guides](blogs/Add_Guides.md)
    - [Add release notes](blogs/Add_Release_Notes.md)
    - [Auto generate docstrings](blogs/Auto_Generate_Docstrings.md)
    - [Basic User Guide](blogs/Basic_User_Guide.md)
    - [Configure social share image](blogs/Configure_Social_Share_Image.md)
    - [Customizing the sidebar](blogs/Customizing_The_Sidebar.md)
    - [Material for MkDocs Customization](blogs/Material_for_MkDocs_Customization.md)
- guides
    - [Add guides](guides/Add_Guides.md)
    - [Add release notes](guides/Add_Release_Notes.md)
    - [Auto generate docstrings](guides/Auto_Generate_Docstrings.md)
    - [Basic User Guide](guides/Basic_User_Guide.md)
    - [Configure social share image](guides/Configure_Social_Share_Image.md)
    - [Customizing the sidebar](guides/Customizing_The_Sidebar.md)
    - [Material for MkDocs Customization](guides/Material_for_MkDocs_Customization.md)
"""
    assert actual == expected, actual

settings.ini created.


pandoc -o README.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  output-file: index.html
  standalone: true
  default-image-extension: png
  
metadata
  title: Material for nbdev
  
Output created: _docs/README.md



../nbs/guides/Material_for_MkDocs_Customization.ipynb, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp7mr9rv2q/nbs/guides/Material_for_MkDocs_Customization.ipynb
../nbs/guides/Customizing_The_Sidebar.ipynb, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp7mr9rv2q/nbs/guides/Customizing_The_Sidebar.ipynb
../nbs/guides/Add_Guides.ipynb, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp7mr9rv2q/nbs/guides/Add_Guides.ipynb
../nbs/guides/Basic_User_Guide.ipynb, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp7mr9rv2q/nbs/guides/Basic_User_Guide.ipynb
../nbs/guides/Add_Release_Notes.ipynb, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp7mr9rv2q/nbs/guides/Add_Release_Notes.ipynb
../nbs/guides/Configure_Social_Share_Image.ipynb, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp7mr9rv2q/nbs/guides/Configure_Social_Share_Image.ipynb
../nbs/guides/Auto_Generate_Docstrings.ipynb, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp7mr9rv2q/nbs/guides/Auto_Generate_Docstrin

[ 1/18] blogs/Material_for_MkDocs_Customization.ipynb
[ 2/18] blogs/Customizing_The_Sidebar.ipynb
[ 3/18] blogs/Add_Guides.ipynb
[ 4/18] blogs/Basic_User_Guide.ipynb
[ 5/18] blogs/Add_Release_Notes.ipynb
[ 6/18] blogs/Configure_Social_Share_Image.ipynb
[ 7/18] blogs/Auto_Generate_Docstrings.ipynb
[ 8/18] Mkdocs.ipynb
[ 9/18] guides/Material_for_MkDocs_Customization.ipynb
[10/18] guides/Customizing_The_Sidebar.ipynb
[11/18] guides/Add_Guides.ipynb
[12/18] guides/Basic_User_Guide.ipynb
[13/18] guides/Add_Release_Notes.ipynb
[14/18] guides/Configure_Social_Share_Image.ipynb
[15/18] guides/Auto_Generate_Docstrings.ipynb
[16/18] api/Mkdocs.ipynb
[17/18] index.ipynb
[18/18] Social_Image_Generator.ipynb

Output created: _docs/index.html



- [Material for nbdev](index.md)
- [Create new](Mkdocs.md)
- [Social_Image_Generator.html](Social_Image_Generator.md)
- api
    - [Create new](api/Mkdocs.md)
- blogs
    - [Add guides](blogs/Add_Guides.md)
    - [Add release notes](blogs/Add_Release_Notes.md)
    - [Auto generate docstrings](blogs/Auto_Generate_Docstrings.md)
    - [Basic User Guide](blogs/Basic_User_Guide.md)
    - [Configure social share image](blogs/Configure_Social_Share_Image.md)
    - [Customizing the sidebar](blogs/Customizing_The_Sidebar.md)
    - [Material for MkDocs Customization](blogs/Material_for_MkDocs_Customization.md)
- guides
    - [Add guides](guides/Add_Guides.md)
    - [Add release notes](guides/Add_Release_Notes.md)
    - [Auto generate docstrings](guides/Auto_Generate_Docstrings.md)
    - [Basic User Guide](guides/Basic_User_Guide.md)
    - [Configure social share image](guides/Configure_Social_Share_Image.md)
    - [Customizing the sidebar](guides/Customizing_The_Sidebar.md)
    - [Material for M

### Build API

In [ ]:
# | export


def _get_submodules(package_name: str) -> List[str]:
    """Get all submodules of a package.

    Args:
        package_name: The name of the package.

    Returns:
        A list of submodules.

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
    """
    # nosemgrep: python.lang.security.audit.non-literal-import.non-literal-import
    m = importlib.import_module(package_name)
    submodules = [
        info.name
        for info in pkgutil.walk_packages(m.__path__, prefix=f"{package_name}.")
    ]
    submodules = [
        x
        for x in submodules
        if not any([name.startswith("_") for name in x.split(".")])
    ]
    return submodules

In [ ]:
submodules = _get_submodules("mkdocs")
submodules

['mkdocs.commands',
 'mkdocs.commands.babel',
 'mkdocs.commands.build',
 'mkdocs.commands.gh_deploy',
 'mkdocs.commands.new',
 'mkdocs.commands.serve',
 'mkdocs.commands.setup',
 'mkdocs.config',
 'mkdocs.config.base',
 'mkdocs.config.config_options',
 'mkdocs.config.defaults',
 'mkdocs.contrib',
 'mkdocs.contrib.search',
 'mkdocs.contrib.search.search_index',
 'mkdocs.exceptions',
 'mkdocs.livereload',
 'mkdocs.localization',
 'mkdocs.plugins',
 'mkdocs.structure',
 'mkdocs.structure.files',
 'mkdocs.structure.nav',
 'mkdocs.structure.pages',
 'mkdocs.structure.toc',
 'mkdocs.tests',
 'mkdocs.tests.base',
 'mkdocs.tests.build_tests',
 'mkdocs.tests.cli_tests',
 'mkdocs.tests.config',
 'mkdocs.tests.config.base_tests',
 'mkdocs.tests.config.config_options_legacy_tests',
 'mkdocs.tests.config.config_options_tests',
 'mkdocs.tests.config.config_tests',
 'mkdocs.tests.gh_deploy_tests',
 'mkdocs.tests.integration',
 'mkdocs.tests.livereload_tests',
 'mkdocs.tests.localization_tests',
 'mkd

In [ ]:
# | export


def _copy_not_found_file_and_get_path(root_path: str, file_prefix: str) -> str:
    """Copy the CLI command found file to the docs directory and return the path to the file.

    Args:
        root_path: The root path of the project
        file_prefix: The prefix of the file to be copied

    Returns:
        The path to the copied file

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
    """
    src_path = get_root_data_path() / f"{file_prefix}_not_found.md"
    if not src_path.exists():
        typer.secho(
            f"Unexpected error: path {src_path.resolve()} does not exists!",
            err=True,
            fg=typer.colors.RED,
        )
        raise typer.Exit(code=1)

    docs_path = Path(root_path) / "mkdocs" / "docs"
    docs_path.mkdir(exist_ok=True, parents=True)
    dst_path = docs_path / f"{file_prefix}_not_found.md"
    shutil.copyfile(src_path, dst_path)

    return (
        f"({dst_path.name})"
        if file_prefix == "changelog"
        else " " * 4 + f"- [Not found]({dst_path.name})"
    )

In [ ]:
with TemporaryDirectory() as d:
    run_nbdev_new(d)
    new(d)

    cli_summary = _copy_not_found_file_and_get_path(d, "cli_commands")
    print(cli_summary)

    _dst_path = Path(d) / "mkdocs" / "docs"
    assert cli_summary == " " * 4 + "- [Not found](cli_commands_not_found.md)"
    assert (_dst_path / "cli_commands_not_found.md").exists()

settings.ini created.


pandoc -o README.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  output-file: index.html
  standalone: true
  default-image-extension: png
  
metadata
  title: Material for nbdev
  


Directory /private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp_k3a94n7/mkdocs created.
File '/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp_k3a94n7/mkdocs/mkdocs.yml' generated.
File '/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp_k3a94n7/mkdocs/summary_template.txt' generated.
    - [Not found](cli_commands_not_found.md)


Output created: _docs/README.md



In [ ]:
# | export


def _generate_api_doc_for_submodule(
    root_path: str, docs_dir_name: str, submodule: str
) -> str:
    """Generate API documentation for a submodule.

    Args:
        root_path: The root path of the project
        docs_dir_name: The name of the docs directory
        submodule: The submodule for which to generate the API documentation

    Returns:
        The path to the generated API documentation

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
    """
    subpath = f"{docs_dir_name}/" + submodule.replace(".", "/") + ".md"
    path = Path(root_path) / "mkdocs" / "docs" / subpath
    path.parent.mkdir(exist_ok=True, parents=True)
    with open(path, "w") as f:
        f.write(f"::: {submodule}")
    subnames = submodule.split(".")
    if len(subnames) > 2:
        return " " * 4 * (len(subnames) - 2) + f"- [{subnames[-1]}]({subpath})"
    else:
        return f"- [{submodule}]({subpath})"


def _generate_api_docs_for_module(root_path: str, module_name: str) -> str:
    """Generate API documentation for a module.

    Args:
        root_path: The root path of the project.
        module_name: The name of the module.

    Returns:
        A string containing the API documentation for the module.

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
    """
    submodules = _get_submodules(module_name)
    docs_dir_name = f"{module_name}_api_docs"
    shutil.rmtree(
        Path(root_path) / "mkdocs" / "docs" / f"{docs_dir_name}", ignore_errors=True
    )

    if len(submodules) == 0:
        submodules = [f"{module_name}"]

    submodule_summary = "\n".join(
        [
            _generate_api_doc_for_submodule(
                root_path=root_path, docs_dir_name=docs_dir_name, submodule=x
            )
            for x in submodules
        ]
    )

    return textwrap.indent(submodule_summary, prefix=" " * 4)

In [ ]:
@contextmanager
def add_tmp_path_to_sys_path(dir_):
    """Add a temporary path to sys.path

    Args:
        dir_ : the path to add to sys.path

    Returns:
        None

    Raises:
        ValueError: If dir_ is None

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
    """
    dir_ = Path(dir_).absolute().resolve(strict=True)
    original_path = sys.path[:]
    sys.path.insert(0, str(dir_))
    try:
        yield
    finally:
        sys.path = original_path


with TemporaryDirectory() as d:
    run_nbdev_new(d)
    settings_path = Path(d) / "settings.ini"
    _root_path = Path(".") if Path("settings.ini").exists() else Path("..")

    copy_guides(_root_path, d)

    new(d)
    _module = "repo"

    with add_tmp_path_to_sys_path(d):
        api_summary = _generate_api_docs_for_module(d, _module)

    print(api_summary)

    # make sure all paths exist
    paths = re.findall("\(.*?\)", api_summary)
    paths = [Path(d) / "mkdocs/docs" / x[1:-1] for x in paths]
    for path in paths:
        assert path.exists(), path
        assert f"{_module}_api_docs" in str(path)

settings.ini created.


pandoc -o README.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  output-file: index.html
  standalone: true
  default-image-extension: png
  
metadata
  title: Material for nbdev
  


../nbs/guides/Material_for_MkDocs_Customization.ipynb, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpuwa9_dg4/nbs/guides/Material_for_MkDocs_Customization.ipynb
../nbs/guides/Customizing_The_Sidebar.ipynb, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpuwa9_dg4/nbs/guides/Customizing_The_Sidebar.ipynb
../nbs/guides/Add_Guides.ipynb, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpuwa9_dg4/nbs/guides/Add_Guides.ipynb
../nbs/guides/Basic_User_Guide.ipynb, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpuwa9_dg4/nbs/guides/Basic_User_Guide.ipynb
../nbs/guides/Add_Release_Notes.ipynb, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpuwa9_dg4/nbs/guides/Add_Release_Notes.ipynb
../nbs/guides/Configure_Social_Share_Image.ipynb, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpuwa9_dg4/nbs/guides/Configure_Social_Share_Image.ipynb
../nbs/guides/Auto_Generate_Docstrings.ipynb, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpuwa9_dg4/nbs/guides/Auto_Generate_Docstrin

Output created: _docs/README.md



In [ ]:
# | export


def _restrict_line_length(s: str, width: int = 80) -> str:
    """Restrict the line length of a string.

    Args:
        s: The string to be processed.
        width: The maximum line length.

    Returns:
        A new string in which each line is less than the specified width.

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
    """
    _s = ""

    for blocks in s.split("\n\n"):
        sub_block = blocks.split("\n  ")
        for line in sub_block:
            line = line.replace("\n", " ")
            line = "\n".join(textwrap.wrap(line, width=width, replace_whitespace=False))
            if len(sub_block) == 1:
                _s += line + "\n\n"
            else:
                _s += "\n" + line + "\n" if line.endswith(":") else " " + line + "\n"
    return _s

In [ ]:
s = """usage: nbdev_mkdocs_new [-h] root_path

Initialize mkdocs project files Creates **mkdocs** directory in the **root_path** directory and populates it with
initial values. You should edit mkdocs.yml file to customize it if needed.

positional arguments:
  root_path

optional arguments:
  -h, --help  show this help message and exit show this help message and exit show this help message and exit
  -h, --help  show this help message and exit
  --port PORT
  --domain DOMAIN
"""

width = 60
doc = _restrict_line_length(s, width)

print(doc)
assert all([len(line) <= width for line in doc.splitlines() if line.strip() != ""])

usage: nbdev_mkdocs_new [-h] root_path

Initialize mkdocs project files Creates **mkdocs** directory
in the **root_path** directory and populates it with initial
values. You should edit mkdocs.yml file to customize it if
needed.


positional arguments:
 root_path

optional arguments:
 -h, --help  show this help message and exit show this help
message and exit show this help message and exit
 -h, --help  show this help message and exit
 --port PORT
 --domain DOMAIN



In [ ]:
# | export


def _generate_cli_doc_for_submodule(
    root_path: str, docs_dir_name: str, cmd: str
) -> str:
    """Generate CLI documentation for a submodule.

    Args:
        root_path: The root path of the project.
        docs_dir_name: The name of the directory where the documentation will be stored.
        cmd: The command to generate documentation for.

    Returns:
        The generated documentation.

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
    """
    cli_app_name = cmd.split("=")[0]
    module_name = cmd.split("=")[1].split(":")[0]
    method_name = cmd.split("=")[1].split(":")[1]

    subpath = f"{docs_dir_name}/{cli_app_name}.md"
    path = Path(root_path) / "mkdocs" / "docs" / subpath
    path.parent.mkdir(exist_ok=True, parents=True)

    try:
        # nosemgrep: python.lang.security.audit.non-literal-import.non-literal-import
        m = importlib.import_module(module_name)
        if isinstance(getattr(m, method_name), typer.Typer):
            cli_doc = generate_cli_doc(module_name, cli_app_name)
        else:
            cmd = f"{cli_app_name} --help"
            cli_doc = (
                # nosemgrep: python.lang.security.audit.subprocess-shell-true.subprocess-shell-true
                subprocess.run(  # nosec: B602:subprocess_popen_with_shell_equals_true
                    cmd,
                    shell=True,
                    stdout=subprocess.PIPE,
                    stderr=subprocess.STDOUT,
                ).stdout.decode("utf-8")
            )

            cli_doc = _restrict_line_length(cli_doc)
            cli_doc = "\n```\n" + cli_doc + "\n```\n"

    except AttributeError as e:
        cli_doc = f"Unable to generate documentation for command. Execution of `{cli_app_name} --help` command failed."

    with open(path, "w") as f:
        f.write(cli_doc)

    return f"- [{cli_app_name}]({subpath})"


def _generate_cli_docs_for_module(root_path: str, module_name: str) -> str:
    """Generate CLI docs for a module.

    Args:
        root_path: The root path of the module
        module_name: The name of the module

    Returns:
        The generated CLI docs

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
    """
    docs_dir_name = f"{module_name}_cli_docs"
    shutil.rmtree(
        Path(root_path) / "mkdocs" / "docs" / f"{docs_dir_name}", ignore_errors=True
    )
    console_scripts = get_value_from_config(root_path, "console_scripts")

    if not console_scripts:
        ret_val = _copy_not_found_file_and_get_path(
            root_path=root_path, file_prefix="cli_commands"
        )
        return ret_val

    submodule_summary = "\n".join(
        [
            _generate_cli_doc_for_submodule(
                root_path=root_path, docs_dir_name=docs_dir_name, cmd=cmd
            )
            for cmd in console_scripts.split("\n")
            if cmd != ""
        ]
    )

    return textwrap.indent(submodule_summary, prefix=" " * 4)

In [ ]:
with TemporaryDirectory() as d:
    run_nbdev_new(d)

    settings_path = Path(d) / "settings.ini"
    shutil.copyfile(Path("..") / "settings.ini", settings_path)

    # update settings.ini file add invalid console script
    updater = ConfigUpdater()
    updater.read(settings_path)
    updater["DEFAULT"]["console_scripts"] = "nbdev_mkdocs=nbdev_mkdocs._cli:_app"
    updater["DEFAULT"]["console_scripts"].append(
        "invalid_cmd=nbdev_mkdocs._cli:_invalid_app"
    )
    updater.update_file()

    new(d)

    _module = "nbdev_mkdocs"
    cli_summary = _generate_cli_docs_for_module(d, _module)
    print(cli_summary)
    expected = """    - [nbdev_mkdocs](nbdev_mkdocs_cli_docs/nbdev_mkdocs.md)
    - [invalid_cmd](nbdev_mkdocs_cli_docs/invalid_cmd.md)"""

    assert cli_summary == expected

    _dst_path = Path(d) / "mkdocs" / "docs"
    assert (_dst_path / "nbdev_mkdocs_cli_docs" / "nbdev_mkdocs.md").exists()
    assert (_dst_path / "nbdev_mkdocs_cli_docs" / "invalid_cmd.md").exists()

    with (_dst_path / "nbdev_mkdocs_cli_docs" / "invalid_cmd.md").open("r") as f:
        contents = f.read()

    print(contents)
    assert (
        contents
        == "Unable to generate documentation for command. Execution of `invalid_cmd --help` command failed."
    )

settings.ini created.


pandoc -o README.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  output-file: index.html
  standalone: true
  default-image-extension: png
  
metadata
  title: Material for nbdev
  


Directory /private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp_65eioa2/mkdocs created.
File '/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp_65eioa2/mkdocs/mkdocs.yml' generated.
File '/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp_65eioa2/mkdocs/summary_template.txt' generated.
    - [nbdev_mkdocs](nbdev_mkdocs_cli_docs/nbdev_mkdocs.md)
    - [invalid_cmd](nbdev_mkdocs_cli_docs/invalid_cmd.md)
Unable to generate documentation for command. Execution of `invalid_cmd --help` command failed.


Output created: _docs/README.md



In [ ]:
with TemporaryDirectory() as d:
    run_nbdev_new(d)
    settings_path = Path(d) / "settings.ini"
    new(d)

    _module = "invalid_module"
    cli_summary = _generate_cli_docs_for_module(d, _module)
    print(cli_summary)

    _dst_path = Path(d) / "mkdocs" / "docs"

    for path in _dst_path.iterdir():
        print(path)

    assert cli_summary == " " * 4 + "- [Not found](cli_commands_not_found.md)"

    assert (_dst_path / "cli_commands_not_found.md").exists()

settings.ini created.


pandoc -o README.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  output-file: index.html
  standalone: true
  default-image-extension: png
  
metadata
  title: Material for nbdev
  


Directory /private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp1ajyyk6g/mkdocs created.
File '/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp1ajyyk6g/mkdocs/mkdocs.yml' generated.
File '/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp1ajyyk6g/mkdocs/summary_template.txt' generated.
    - [Not found](cli_commands_not_found.md)
/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp1ajyyk6g/mkdocs/docs/cli_commands_not_found.md


Output created: _docs/README.md



In [ ]:
with TemporaryDirectory() as d:
    run_nbdev_new(d)

    settings_path = Path(d) / "settings.ini"
    shutil.copyfile(Path("..") / "settings.ini", settings_path)

    new(d)

    _module = "nbdev_mkdocs"
    cli_summary = _generate_cli_docs_for_module(d, _module)
    print(cli_summary)

    # make sure all paths and content exist
    paths = re.findall("\(.*?\)", cli_summary)
    paths = [Path(d) / "mkdocs/docs" / x[1:-1] for x in paths]
    for path in paths:
        assert path.exists(), path
        assert f"{_module}_cli_docs" in str(path)

settings.ini created.


pandoc -o README.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  output-file: index.html
  standalone: true
  default-image-extension: png
  
metadata
  title: Material for nbdev
  


Directory /private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp_pm4h45j/mkdocs created.
File '/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp_pm4h45j/mkdocs/mkdocs.yml' generated.
File '/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp_pm4h45j/mkdocs/summary_template.txt' generated.
    - [nbdev_mkdocs](nbdev_mkdocs_cli_docs/nbdev_mkdocs.md)


Output created: _docs/README.md



In [ ]:
# | export


def _copy_change_log_if_exists(root_path: str, docs_path: Union[Path, str]) -> str:
    """Copy the CHANGELOG.md file to the docs folder if it's not already present.

    Args:
        root_path: The root path of the project.
        docs_path: The path to the docs folder.

    Returns:
        The path to the copied CHANGELOG.md file.

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
    """
    source_change_log_path = Path(root_path) / "CHANGELOG.md"
    dst_change_log_path = Path(docs_path) / "CHANGELOG.md"

    if source_change_log_path.exists():
        shutil.copy(source_change_log_path, dst_change_log_path)
        changelog = "(CHANGELOG.md)"
    else:
        changelog = _copy_not_found_file_and_get_path(
            root_path=root_path, file_prefix="changelog"
        )

    return changelog

In [ ]:
with TemporaryDirectory() as d:

    run_nbdev_new(d)

    settings_path = Path(d) / "settings.ini"
    shutil.copyfile(Path("..") / "settings.ini", settings_path)

    new(d)

    change_log = _copy_change_log_if_exists(d, f"{d}/mkdocs/docs")

    print(f"change_log={change_log}")
    assert change_log == "(changelog_not_found.md)"

    change_log_path = Path(d) / "CHANGELOG.md"
    with open(change_log_path, "w") as f:
        f.write("CHANGELOG")

    (Path(d) / "mkdocs" / "docs").mkdir(exist_ok=True, parents=True)
    change_log = _copy_change_log_if_exists(d, f"{d}/mkdocs/docs")

    print(f"change_log={change_log}")
    assert change_log == "(CHANGELOG.md)"

settings.ini created.


pandoc -o README.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  output-file: index.html
  standalone: true
  default-image-extension: png
  
metadata
  title: Material for nbdev
  


Directory /private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpjrhug8y4/mkdocs created.
File '/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpjrhug8y4/mkdocs/mkdocs.yml' generated.
File '/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpjrhug8y4/mkdocs/summary_template.txt' generated.
change_log=(changelog_not_found.md)
change_log=(CHANGELOG.md)


Output created: _docs/README.md



### Bringing it all together

In [ ]:
# | export


def _build_summary(
    root_path: str,
    module: str,
) -> None:
    # create docs_path if needed
    """Create a summary navigation file for generating navigation that is compatible with mkdocs.

    Args:
        root_path: The root path of the project.
        module: The module to generate the API documentation for.

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
    """
    docs_path = Path(root_path) / "mkdocs" / "docs"
    docs_path.mkdir(exist_ok=True)

    # copy README.md as index.md
    shutil.copy(Path(root_path) / "README.md", docs_path / "index.md")

    # generate markdown files
    _generate_markdown_from_files(root_path)

    # copy images to docs dir and update path in generated markdown files
    _copy_images_to_docs_dir(root_path)

    # generates sidebar navigation
    sidebar = _generate_summary_for_sidebar(root_path)

    # generate API
    api = _generate_api_docs_for_module(root_path, module)

    # generate CLI
    cli = _generate_cli_docs_for_module(root_path, module)

    # copy CHANGELOG.md as CHANGELOG.md is exists
    changelog = _copy_change_log_if_exists(root_path, docs_path)

    # read summary template from file
    with open(Path(root_path) / "mkdocs" / "summary_template.txt") as f:
        summary_template = f.read()

    summary = summary_template.format(
        sidebar=sidebar, api=api, cli=cli, changelog=changelog
    )
    summary = "\n".join(
        [l for l in [l.rstrip() for l in summary.split("\n")] if l != ""]
    )

    with open(docs_path / "SUMMARY.md", mode="w") as f:
        f.write(summary)

In [ ]:
_test_summary_template = """{sidebar}
- Reference
{api}
- Command line
{cli}
- [Change log]{changelog}
"""

with TemporaryDirectory() as d:
    run_nbdev_new(d)

    settings_path = Path(d) / "settings.ini"
    for fname in ["settings.ini", "README.md", "CHANGELOG.md"]:
        shutil.copyfile(Path("..") / fname, Path(d) / fname)

    updater = ConfigUpdater()
    updater.read(Path(d) / "settings.ini")
    updater["DEFAULT"]["custom_sidebar"] = False
    updater.update_file()

    copy_guides(_root_path, d)
    for f in ["Mkdocs.ipynb", "Social_Image_Generator.ipynb"]:
        shutil.copyfile(_root_path / "nbs" / f, Path(d) / "nbs" / f)

    shutil.copytree((Path(d) / "nbs" / "guides"), (Path(d) / "nbs" / "blogs"))

    new(d)

    _summary_template_path = Path(d) / "mkdocs" / "summary_template.txt"
    _summary_template_path.unlink()
    with _summary_template_path.open("w", encoding="utf-8") as f:
        f.write(_test_summary_template)

    _build_summary(d, "repo")

    with open(Path(d) / "mkdocs/docs/SUMMARY.md") as f:
        summary = f.read()

    print(summary)
    assert "- [Material for nbdev](index.md)" in summary
    assert "- [Change log](CHANGELOG.md)" in summary
    assert "- Reference\n" in summary
    assert "- Command line\n" in summary

settings.ini created.


pandoc -o README.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  output-file: index.html
  standalone: true
  default-image-extension: png
  
metadata
  title: Material for nbdev
  


../nbs/guides/Material_for_MkDocs_Customization.ipynb, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpb3aejoe0/nbs/guides/Material_for_MkDocs_Customization.ipynb
../nbs/guides/Customizing_The_Sidebar.ipynb, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpb3aejoe0/nbs/guides/Customizing_The_Sidebar.ipynb
../nbs/guides/Add_Guides.ipynb, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpb3aejoe0/nbs/guides/Add_Guides.ipynb
../nbs/guides/Basic_User_Guide.ipynb, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpb3aejoe0/nbs/guides/Basic_User_Guide.ipynb
../nbs/guides/Add_Release_Notes.ipynb, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpb3aejoe0/nbs/guides/Add_Release_Notes.ipynb
../nbs/guides/Configure_Social_Share_Image.ipynb, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpb3aejoe0/nbs/guides/Configure_Social_Share_Image.ipynb
../nbs/guides/Auto_Generate_Docstrings.ipynb, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpb3aejoe0/nbs/guides/Auto_Generate_Docstrin

Output created: _docs/README.md

pandoc -o Mkdocs.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  standalone: true
  default-image-extension: png
  
Output created: _docs/Mkdocs.md

pandoc -o index.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  output-file: index.html
  standalone: true
  default-image-extension: png
  
metadata
  title: Material for nbdev
  
Output created: _docs/index.md

pandoc -o Social_Image_Generator.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  standalone: true
  default-image-extension: png
  
Output created: _docs/Social_Image_Generator.md

pandoc -o ../Material_for_MkDocs_Customization.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+

- [Material for nbdev](index.md)
- [Create new](Mkdocs.md)
- [Social_Image_Generator.html](Social_Image_Generator.md)
- blogs
    - [Add guides](blogs/Add_Guides.md)
    - [Add release notes](blogs/Add_Release_Notes.md)
    - [Auto generate docstrings](blogs/Auto_Generate_Docstrings.md)
    - [Basic User Guide](blogs/Basic_User_Guide.md)
    - [Configure social share image](blogs/Configure_Social_Share_Image.md)
    - [Customizing the sidebar](blogs/Customizing_The_Sidebar.md)
    - [Material for MkDocs Customization](blogs/Material_for_MkDocs_Customization.md)
- guides
    - [Add guides](guides/Add_Guides.md)
    - [Add release notes](guides/Add_Release_Notes.md)
    - [Auto generate docstrings](guides/Auto_Generate_Docstrings.md)
    - [Basic User Guide](guides/Basic_User_Guide.md)
    - [Configure social share image](guides/Configure_Social_Share_Image.md)
    - [Customizing the sidebar](guides/Customizing_The_Sidebar.md)
    - [Material for MkDocs Customization](guides/Material_for

### Copy CNAME if needed

In [ ]:
# | export


def _copy_cname_if_needed(root_path: str) -> None:
    """Copy the CNAME file to mkdocs/docs/CNAME if it's not already present.

    Args:
        root_path: The root path of the project

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
    """
    cname_path = Path(root_path) / "CNAME"
    dst_path = Path(root_path) / "mkdocs" / "docs" / "CNAME"
    if cname_path.exists():
        dst_path.parent.mkdir(exist_ok=True, parents=True)
        shutil.copyfile(cname_path, dst_path)
        typer.secho(
            f"File '{cname_path.resolve()}' copied to '{dst_path.resolve()}'.",
        )
    else:
        typer.secho(
            f"File '{cname_path.resolve()}' not found, skipping copying..",
        )

In [ ]:
for has_cname in [True, False]:
    with TemporaryDirectory() as d:
        settings_path = Path(d) / "settings.ini"
        for fname in ["settings.ini", "README.md"] + ["CNAME"] if has_cname else []:
            shutil.copyfile(Path("..") / fname, Path(d) / fname)

        _copy_cname_if_needed(d)
        if has_cname:
            assert (Path(d) / "mkdocs" / "docs" / "CNAME").exists()
        else:
            assert not (Path(d) / "mkdocs" / "docs" / "CNAME").exists()

File '/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpwqkv37wu/CNAME' copied to '/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpwqkv37wu/mkdocs/docs/CNAME'.
File '/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmp_elcsm8f/CNAME' not found, skipping copying..


In [ ]:
# | export


def _copy_docs_overrides(root_path: str) -> None:
    """Copy the docs_overrides directory to the mkdocs/docs/overrides directory.

    Args:
        root_path: The root path of the project.

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
    """
    src_path = Path(root_path) / "mkdocs" / "docs_overrides"
    dst_path = Path(root_path) / "mkdocs" / "docs" / "overrides"

    if not src_path.exists():
        typer.secho(
            f"Unexpected error: path {src_path.resolve()} does not exists!",
            err=True,
            fg=typer.colors.RED,
        )
        raise typer.Exit(code=1)

    shutil.rmtree(dst_path, ignore_errors=True)
    shutil.copytree(src_path, dst_path)

In [ ]:
with TemporaryDirectory() as d:
    run_nbdev_new(d)

    settings_path = Path(d) / "settings.ini"
    assert settings_path.exists()

    new(d)

    assert (Path(d) / "mkdocs" / "docs_overrides").exists()
    assert (Path(d) / "mkdocs" / "docs_overrides" / "css").exists()
    assert (Path(d) / "mkdocs" / "docs_overrides" / "js").exists()
    assert (Path(d) / "mkdocs" / "docs_overrides" / "images").exists()
    assert (Path(d) / "mkdocs" / "docs_overrides" / "css" / "extra.css").exists()
    assert (Path(d) / "mkdocs" / "docs_overrides" / "js" / "extra.js").exists()

    _copy_docs_overrides(d)

    assert (Path(d) / "mkdocs" / "docs" / "overrides").exists()
    assert (Path(d) / "mkdocs" / "docs" / "overrides" / "css").exists()
    assert (Path(d) / "mkdocs" / "docs" / "overrides" / "js").exists()
    assert (Path(d) / "mkdocs" / "docs" / "overrides" / "images").exists()
    assert (Path(d) / "mkdocs" / "docs" / "overrides" / "css" / "extra.css").exists()
    assert (Path(d) / "mkdocs" / "docs" / "overrides" / "js" / "extra.js").exists()

settings.ini created.


pandoc -o README.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  output-file: index.html
  standalone: true
  default-image-extension: png
  
metadata
  title: Material for nbdev
  


Directory /private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpqpsqfxhj/mkdocs created.
File '/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpqpsqfxhj/mkdocs/mkdocs.yml' generated.
File '/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpqpsqfxhj/mkdocs/summary_template.txt' generated.


Output created: _docs/README.md



In [ ]:
# | export


def nbdev_mkdocs_docs(root_path: str, refresh_quarto_settings: bool = False) -> None:
    """Prepare mkdocs documentation

    Args:
        root_path: The root path of the project
        refresh_quarto_settings: Flag to refresh quarto yml file. This flag should be set to `True`
            if this function is called directly without calling prepare.

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
    """
    with set_cwd(root_path):

        if refresh_quarto_settings:
            refresh_quarto_yml()

        _copy_cname_if_needed(root_path)

        _copy_docs_overrides(root_path)

        lib_name = get_value_from_config(root_path, "lib_name")
        lib_path = get_value_from_config(root_path, "lib_path")

        _build_summary(root_path, lib_path)

        cmd = f"mkdocs build -f \"{(Path(root_path) / 'mkdocs' / 'mkdocs.yml').resolve()}\""
        _sprun(cmd)



def prepare(root_path: str, no_test: bool = False) -> None:
    """Prepare mkdocs for serving

    Args:
        root_path: The root path of the project
        no_test: If set to False, the unit tests will be run, else they will be skipped

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
    """
    with set_cwd(root_path):

        if no_test:
            nbdev_export.__wrapped__()
            refresh_quarto_yml()
            nbdev_readme.__wrapped__(chk_time=True)
        else:
            cmd = "nbdev_prepare"
            _sprun(cmd)

    nbdev_mkdocs_docs(root_path)


In [ ]:
with TemporaryDirectory() as d:
    run_nbdev_new(d)

    settings_path = Path(d) / "settings.ini"
    assert settings_path.exists()

    _root_path = Path(".") if Path("settings.ini").exists() else Path("..")
    copy_guides(_root_path, d)

    for f in ["CNAME", "nbs/Docstring.ipynb"]:
        shutil.copyfile(_root_path / f, Path(d) / f)

    new(d)
    #     !cat {d}/mkdocs/mkdocs.yml

    prepare(d)

    assert (Path(d) / "mkdocs" / "docs" / "CNAME").exists()
    assert (Path(d) / "mkdocs" / "docs" / "SUMMARY.md").exists()
    assert (Path(d) / "mkdocs" / "docs" / "index.md").exists()
    assert (Path(d) / "mkdocs" / "docs" / "guides").exists()
# #     !ls {d}/mkdocs/docs

settings.ini created.


pandoc -o README.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  output-file: index.html
  standalone: true
  default-image-extension: png
  
metadata
  title: Material for nbdev
  


../nbs/guides/Material_for_MkDocs_Customization.ipynb, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpcbcqcm_d/nbs/guides/Material_for_MkDocs_Customization.ipynb
../nbs/guides/Customizing_The_Sidebar.ipynb, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpcbcqcm_d/nbs/guides/Customizing_The_Sidebar.ipynb
../nbs/guides/Add_Guides.ipynb, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpcbcqcm_d/nbs/guides/Add_Guides.ipynb
../nbs/guides/Basic_User_Guide.ipynb, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpcbcqcm_d/nbs/guides/Basic_User_Guide.ipynb
../nbs/guides/Add_Release_Notes.ipynb, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpcbcqcm_d/nbs/guides/Add_Release_Notes.ipynb
../nbs/guides/Configure_Social_Share_Image.ipynb, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpcbcqcm_d/nbs/guides/Configure_Social_Share_Image.ipynb
../nbs/guides/Auto_Generate_Docstrings.ipynb, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpcbcqcm_d/nbs/guides/Auto_Generate_Docstrin

Output created: _docs/README.md



File '/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpcbcqcm_d/CNAME' copied to '/private/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpcbcqcm_d/mkdocs/docs/CNAME'.


pandoc -o Docstring.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  output-file: docstring.html
  standalone: true
  default-image-extension: png
  
metadata
  title: Docstring helpers
  
Output created: _docs/Docstring.md

pandoc -o index.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  output-file: index.html
  standalone: true
  default-image-extension: png
  
metadata
  title: Material for nbdev
  
Output created: _docs/index.md

pandoc -o ../Material_for_MkDocs_Customization.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  output-file: material_for_mkdocs_customization.html
  standalone: true
  default-image-extension: png
  
metadata
  title: Material for MkDocs Customization
  
Output created: ../_docs/Material_for_MkDocs_Cu

## Preview

In [ ]:
# | export


def preview(root_path: str, port: Optional[int] = None) -> None:
    """Preview the mkdocs documentation.

    Args:
        root_path: The root path of the documentation.
        port: The port to serve the documentation on.

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)
    """
    with set_cwd(root_path):
        prepare(root_path=root_path, no_test=True)

        cmd = f"mkdocs serve -f {root_path}/mkdocs/mkdocs.yml -a 0.0.0.0"
        if port:
            cmd = cmd + f":{port}"

        with subprocess.Popen(  # nosec B603:subprocess_without_shell_equals_true
            shlex.split(cmd),
            stdout=subprocess.PIPE,
            bufsize=1,
            text=True,
            universal_newlines=True,
        ) as p:
            for line in p.stdout:  # type: ignore
                print(line, end="")

        if p.returncode != 0:
            typer.secho(
                f"Command cmd='{cmd}' failed!",
                err=True,
                fg=typer.colors.RED,
            )
            raise typer.Exit(6)


In [ ]:
# | notest

_test_summary_template = """{sidebar}
- Reference
{api}
- Command line
{cli}
- [Change log]{changelog}
"""

with TemporaryDirectory() as d:
    run_nbdev_new(d)

    settings_path = Path(d) / "settings.ini"
    assert settings_path.exists()
    _root_path = Path(".") if Path("settings.ini").exists() else Path("..")

    copy_guides(_root_path, d)
    shutil.copytree((Path(d) / "nbs" / "guides"), (Path(d) / "nbs" / "blogs"))

    new(d)

    _summary_template_path = Path(d) / "mkdocs" / "summary_template.txt"
    _summary_template_path.unlink()
    with _summary_template_path.open("w", encoding="utf-8") as f:
        f.write(_test_summary_template)

    preview(d, port=4000)

settings.ini created.


pandoc -o README.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  output-file: index.html
  standalone: true
  default-image-extension: png
  
metadata
  title: Material for nbdev
  


../nbs/guides/Material_for_MkDocs_Customization.ipynb, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpv_7pd600/nbs/guides/Material_for_MkDocs_Customization.ipynb
../nbs/guides/Customizing_The_Sidebar.ipynb, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpv_7pd600/nbs/guides/Customizing_The_Sidebar.ipynb
../nbs/guides/Add_Guides.ipynb, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpv_7pd600/nbs/guides/Add_Guides.ipynb
../nbs/guides/Basic_User_Guide.ipynb, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpv_7pd600/nbs/guides/Basic_User_Guide.ipynb
../nbs/guides/Add_Release_Notes.ipynb, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpv_7pd600/nbs/guides/Add_Release_Notes.ipynb
../nbs/guides/Configure_Social_Share_Image.ipynb, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpv_7pd600/nbs/guides/Configure_Social_Share_Image.ipynb
../nbs/guides/Auto_Generate_Docstrings.ipynb, /var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpv_7pd600/nbs/guides/Auto_Generate_Docstrin

Output created: _docs/README.md

pandoc -o index.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  output-file: index.html
  standalone: true
  default-image-extension: png
  
metadata
  title: Material for nbdev
  
Output created: _docs/index.md

pandoc -o ../Material_for_MkDocs_Customization.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  output-file: material_for_mkdocs_customization.html
  standalone: true
  default-image-extension: png
  
metadata
  title: Material for MkDocs Customization
  
Output created: ../_docs/Material_for_MkDocs_Customization.md

pandoc -o ../Customizing_The_Sidebar.md
  to: >-
    commonmark+autolink_bare_uris+emoji+footnotes+gfm_auto_identifiers+pipe_tables+strikeout+task_lists+tex_math_dollars
  output-file: customizing_the_sidebar.html
  standalone: true
  default-image-extension: pn

WARNING  -  Documentation file 'index.md' contains a link to 'images/nbs/images/nbdev_mkdocs_banner_img.png' which is not found in the documentation files.
INFO     -  Documentation built in 0.55 seconds
INFO     -  [14:29:49] Watching paths for changes: '/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpv_7pd600/mkdocs/docs', '/var/folders/6n/3rjds7v52cd83wqkd565db0h0000gn/T/tmpv_7pd600/mkdocs/mkdocs.yml'
INFO     -  [14:29:49] Serving on http://0.0.0.0:4000/repo/
WARNING  -  [14:29:53] "GET /repo/images/nbs/images/nbdev_mkdocs_banner_img.png HTTP/1.1" code 404
INFO     -  [14:29:53] Browser connected: http://0.0.0.0:4000/repo/
